In [1]:
import pandas as pd
import numpy as np
import pathlib
import random
import re

RANDOM_SEED = 19439
random.seed(RANDOM_SEED)

## 테스트셋 재처리

SentencePiece와 BPE는 훈련 셋에서 학습한 모델로 테스트셋을 토큰화하고 처리해야 한다.
지금까지는 잘못 수행하고 있었다.

## 전라 방언

### BPE

In [2]:
df = pd.read_csv("../datas/output/jeonla_dialect_bpe_integration.csv", index_col=0)
df

,방언,표준어,연령대,출처 파일,길이,셋
0,ë§īìĥģ ĠìĻĦìłĦ ĠëıĪìĿ´ Ġê¸īíķĺê²Į ĠíķĦìļĶíķľ Ġ...,ë§īìĥģ ĠìĻĦìłĦ ĠëıĪìĿ´ Ġê¸īíķĺê²Į ĠíķĦìļĶíķľ Ġ...,20대,DJSX20002683.json,15,train
1,ì¢ĭìĿĢ Ġì¶ĶìĸµìĿ´ ĠìŀĪìĸ´ê°Ģì§Ģê³ł Ġëĭ¤ìĭľ ĠíĤ...,ì¢ĭìĿĢ Ġì¶ĶìĸµìĿ´ ĠìŀĪìĸ´ê°Ģì§Ģê³ł Ġëĭ¤ìĭľ ĠíĤ...,30대,DJDG20002125.json,8,train
2,ì«Į Ġë³´ë©´ ĠìĿ¸ìłľ ĠìĦ¸ê³Ħ ĠìĿ¸ìłľ,ì¡°ê¸Ī Ġë³´ë©´ ĠìĿ¸ìłľ ĠìĦ¸ê³Ħ ĠìĿ¸ìłľ,30대,DJSX20004304.json,5,train
3,ê°Ģìĭľ ëĤĺëĵ¤ ĠìŀĪìľ¼ëĭĪê¹Į ĠëŃĲ ~ ĠìĿ¼ë¶ĢëŁ¬ ...,ê³Ħ ì§ĳ ìķĦìĿ´ëĵ¤ ĠìŀĪìľ¼ëĭĪê¹Į ĠëŃĲ ~ ĠìĿ¼ë¶Ģ...,20대,DJDG20001833.json,12,train
4,ê±°ê¸°ìĦľ Ġìĸ´ ĠìłĢ ĠìĥĪëģ¼ ĠìĻľ Ġë¯¸ëĭĪ ìĸ¸ Ġ...,ê±°ê¸°ìĦľ Ġìĸ´ ĠìłĢ ĠìĥĪëģ¼ ĠìĻľ Ġë¯¸ëĭĪ ìĸ¸ Ġ...,20대,DJDD20000034.json,12,train
...,...,...,...,...,...,...
275120,ëįĶ ĠëĤĺìķĦ ê°ĢìĦľ ĠëĤĺì¤ĳìĹĲëĬĶ ĠíķĺëĬĺ ĠëıĦì...,ëįĶ ĠëĤĺìķĦ ê°ĢìĦľ ĠëĤĺì¤ĳìĹĲëĬĶ ĠíķĺëĬĺ ĠëıĦì...,30대,DJSX20003150.json,8,test
275123,ìĸ´ ĠíĥĿìĭľ ê°Ģ ĠìŀĪìĿĦ Ġê±° Ġê°Ļìķł .,ìĸ´ ĠíĥĿìĭľ ê°Ģ ĠìŀĪìĿĦ Ġê±° Ġê°Ļìķł .,30대,DJSX20003150.json,5,test
275125,ìļ°ë¦¬ ëĤĺ ëĿ¼ëıĦ ĠìĻĦìłĦ ĠìŀĲìľ¨ì£¼íĸī ì°¨ë¥¼...,ìļ°ë¦¬ ëĤĺ ëĿ¼ëıĦ ĠìĻĦìłĦ ĠìŀĲìľ¨ì£¼íĸī ì°¨ë¥¼...,30대,DJSX20003150.json,12,test
275126,ëĮĢ ë°ķ ìĿ´ëĭ¤ ĠìĿ´ìłľ Ġì¹ł ĠëħĦ ĠëĤ¨ìķĺ ëĦ¤ Ġ...,ëĮĢ ë°ķ ìĿ´ëĭ¤ ĠìĿ´ìłľ Ġì¹ł ĠëħĦ ĠëĤ¨ìķĺ ëĦ¤ Ġ...,30대,DJSX20003150.json,7,test


In [4]:
from tokenizers import ByteLevelBPETokenizer

bpe_tokenizer = ByteLevelBPETokenizer()
bpe_tokenizer.train(
    files="../datas/output/jeonla_dialect_data_text_sampled.txt",
)
bpe_tokenizer.save_model("./")

['./vocab.json', './merges.txt']

In [5]:
df = df[df["셋"] == "test"]
df

,방언,표준어,연령대,출처 파일,길이,셋
0,ëĤĺëĬĶ ĠíļĮìĤ¬ ĠíķĻêµĲìĿĺ ĠìĿ´ìķ¼ê¸°ë¥¼ Ġíķĺê³...,ëĤĺëĬĶ ĠíļĮìĤ¬ ĠíķĻêµĲìĿĺ ĠìĿ´ìķ¼ê¸°ë¥¼ Ġíķĺê³...,20대,DJDD20000247.json,14,test
2,ëıĻ ì°½ Ġì¹ľêµ¬ëĵ¤ìĿĢ ĠìĿ´ìłľ Ġì´Īëĵ±íķĻêµĲ Ġë...,ëıĻ ì°½ Ġì¹ľêµ¬ëĵ¤ìĿĢ ĠìĿ´ìłľ Ġì´Īëĵ±íķĻêµĲ Ġë...,20대,DJDD20000247.json,7,test
3,ìĿĮ Ġ Ġì¹ľêµ¬ëĵ¤ìĿĢ Ġê±°ìĿĺ ĠìĿ´ìłľ Ġì´Īëĵ±íķĻ...,ìĿĮ Ġ Ġì¹ľêµ¬ëĵ¤ìĿĢ Ġê±°ìĿĺ ĠìĿ´ìłľ Ġì´Īëĵ±íķĻ...,20대,DJDD20000247.json,5,test
4,ìĿ´ëłĩê²Į Ġì¹ľêµ¬ìĿ¸ Ġìķł ĠìŀĪê³ł Ġì¤ĳíķĻêµĲ ì...,ìĿ´ëłĩê²Į Ġì¹ľêµ¬ìĿ¸ Ġìķł ĠìŀĪê³ł Ġì¤ĳíķĻêµĲ ì...,20대,DJDD20000247.json,14,test
5,ì¹ľêµ¬ ëĵ¤ëıĦ ĠìĿ´ìłľ Ġì ´¤ ìĬ¹ ĠìķĦ Ġì´Īëĵ±íķ...,ì¹ľêµ¬ ëĵ¤ëıĦ ĠìĿ´ìłľ Ġì ´¤ ìĬ¹ ĠìķĦ Ġì´Īëĵ±íķ...,20대,DJDD20000247.json,13,test
...,...,...,...,...,...,...
275120,ëįĶ ĠëĤĺìķĦ ê°ĢìĦľ ĠëĤĺì¤ĳìĹĲëĬĶ ĠíķĺëĬĺ ĠëıĦì...,ëįĶ ĠëĤĺìķĦ ê°ĢìĦľ ĠëĤĺì¤ĳìĹĲëĬĶ ĠíķĺëĬĺ ĠëıĦì...,30대,DJSX20003150.json,8,test
275123,ìĸ´ ĠíĥĿìĭľ ê°Ģ ĠìŀĪìĿĦ Ġê±° Ġê°Ļìķł .,ìĸ´ ĠíĥĿìĭľ ê°Ģ ĠìŀĪìĿĦ Ġê±° Ġê°Ļìķł .,30대,DJSX20003150.json,5,test
275125,ìļ°ë¦¬ ëĤĺ ëĿ¼ëıĦ ĠìĻĦìłĦ ĠìŀĲìľ¨ì£¼íĸī ì°¨ë¥¼...,ìļ°ë¦¬ ëĤĺ ëĿ¼ëıĦ ĠìĻĦìłĦ ĠìŀĲìľ¨ì£¼íĸī ì°¨ë¥¼...,30대,DJSX20003150.json,12,test
275126,ëĮĢ ë°ķ ìĿ´ëĭ¤ ĠìĿ´ìłľ Ġì¹ł ĠëħĦ ĠëĤ¨ìķĺ ëĦ¤ Ġ...,ëĮĢ ë°ķ ìĿ´ëĭ¤ ĠìĿ´ìłľ Ġì¹ł ĠëħĦ ĠëĤ¨ìķĺ ëĦ¤ Ġ...,30대,DJSX20003150.json,7,test


In [11]:
bpe_tokenizer.from_file("../model_storage/stan-JL_bpe/test_vocab.json", "../model_storage/stan-JL_bpe/test_merges.txt")
for i in range(110459):
    dia = df.iloc[i, 0]
    token_ids = [bpe_tokenizer.token_to_id(token) for token in dia.split()]
    df.iloc[i, 0] = bpe_tokenizer.decode(token_ids)
    
    stan = df.iloc[i, 1]
    token_ids = [bpe_tokenizer.token_to_id(token) for token in stan.split()]
    df.iloc[i, 1] = bpe_tokenizer.decode(token_ids)

df

,방언,표준어,연령대,출처 파일,길이,셋
0,나는 회사 학교의 이야기를 하고 싶어 형은 있잖아요 친구들 같은 거 다 동창 친구들...,나는 회사 학교의 이야기를 하고 싶어 형은 있잖아요 친구들 같은 거 다 동창 친구들...,20대,DJDD20000247.json,14,test
2,동창 친구들은 이제 초등학교 때부터 친구인 거예요?,동창 친구들은 이제 초등학교 때부터 친구인 거예요?,20대,DJDD20000247.json,7,test
3,음 친구들은 거의 이제 초등학교?,음 친구들은 거의 이제 초등학교?,20대,DJDD20000247.json,5,test
4,이렇게 친구인 애 있고 중학교에도 친구들 있고 한데 근데 형은 솔직히 말하면 그거 있잖아,이렇게 친구인 애 있고 중학교에도 친구들 있고 한데 근데 형은 솔직히 말하면 그거 있잖아,20대,DJDD20000247.json,14,test
5,친구들도 이제 촤승 아 초등학교 중학교 이런 것보다는 고등학교에서 진짜 친구 나뉜다...,친구들도 이제 촤승 아 초등학교 중학교 이런 것보다는 고등학교에서 진짜 친구 나뉜다...,20대,DJDD20000247.json,13,test
...,...,...,...,...,...,...
275120,더 나아가서 나중에는 하늘 도시 날아다닐 것 같애.,더 나아가서 나중에는 하늘 도시 날아다닐 것 같애.,30대,DJSX20003150.json,8,test
275123,어 택시가 있을 거 같애.,어 택시가 있을 거 같애.,30대,DJSX20003150.json,5,test
275125,우리나라도 완전 자율주행차를 이 천 이 십 칠 년에 최초로 상용화한다고 발표했대.,우리나라도 완전 자율주행차를 이 천 이 십 칠 년에 최초로 상용화한다고 발표했대.,30대,DJSX20003150.json,12,test
275126,대박이다 이제 칠 년 남았네 칠 년,대박이다 이제 칠 년 남았네 칠 년,30대,DJSX20003150.json,7,test


In [12]:
bpe_tokenizer.from_file("./vocab.json", "./merges.txt")

Tokenizer(vocabulary_size=30000, model=ByteLevelBPE, add_prefix_space=False, lowercase=False, dropout=None, unicode_normalizer=None, continuing_subword_prefix=None, end_of_word_suffix=None, trim_offsets=False)

In [13]:
code_result_dialect = bpe_tokenizer.encode_batch(df["방언"])
code_result_standard = bpe_tokenizer.encode_batch(df["표준어"])

In [14]:
df.loc[df.index, "방언"] = [' '.join(res.tokens) for res in code_result_dialect]
df["방언"]

0         ëĤĺëĬĶ ĠíļĮìĤ¬ ĠíķĻêµĲìĿĺ ĠìĿ´ìķ¼ê¸°ë¥¼ Ġíķĺê³...
2         ëıĻ ì°½ Ġì¹ľêµ¬ëĵ¤ìĿĢ ĠìĿ´ìłľ Ġì´Īëĵ±íķĻêµĲ Ġë...
3         ìĿĮ Ġ Ġì¹ľêµ¬ëĵ¤ìĿĢ Ġê±°ìĿĺ ĠìĿ´ìłľ Ġì´Īëĵ±íķĻ...
4         ìĿ´ëłĩê²Į Ġì¹ľêµ¬ìĿ¸ Ġìķł ĠìŀĪê³ł Ġì¤ĳíķĻêµĲ ì...
5         ì¹ľêµ¬ ëĵ¤ëıĦ ĠìĿ´ìłľ Ġì ´¤ ìĬ¹ ĠìķĦ Ġì´Īëĵ±íķ...
                                ...                        
275120    ëįĶ ĠëĤĺìķĦ ê°ĢìĦľ ĠëĤĺì¤ĳìĹĲëĬĶ ĠíķĺëĬĺ ĠëıĦì...
275123               ìĸ´ ĠíĥĿìĭľ ê°Ģ ĠìŀĪìĿĦ Ġê±° Ġê°Ļìķł .
275125    ìļ°ë¦¬ ëĤĺ ëĿ¼ëıĦ ĠìĻĦìłĦ ĠìŀĲìľ¨ì£¼íĸī ì°¨ë¥¼...
275126    ëĮĢ ë°ķ ìĿ´ëĭ¤ ĠìĿ´ìłľ Ġì¹ł ĠëħĦ ĠëĤ¨ìķĺ ëĦ¤ Ġ...
275134    ê·¼ëį° Ġë°ľíĳľë¥¼ Ġíķľëĭ¤ê³ł Ġíķ´ìĦľ ĠìĿ´ê²Į ĠëĶ±
Name: 방언, Length: 110459, dtype: object

In [15]:
df.loc[df.index, "표준어"] = [' '.join(res.tokens) for res in code_result_standard]
df["표준어"]

0         ëĤĺëĬĶ ĠíļĮìĤ¬ ĠíķĻêµĲìĿĺ ĠìĿ´ìķ¼ê¸°ë¥¼ Ġíķĺê³...
2         ëıĻ ì°½ Ġì¹ľêµ¬ëĵ¤ìĿĢ ĠìĿ´ìłľ Ġì´Īëĵ±íķĻêµĲ Ġë...
3         ìĿĮ Ġ Ġì¹ľêµ¬ëĵ¤ìĿĢ Ġê±°ìĿĺ ĠìĿ´ìłľ Ġì´Īëĵ±íķĻ...
4         ìĿ´ëłĩê²Į Ġì¹ľêµ¬ìĿ¸ Ġìķł ĠìŀĪê³ł Ġì¤ĳíķĻêµĲ ì...
5         ì¹ľêµ¬ ëĵ¤ëıĦ ĠìĿ´ìłľ Ġì ´¤ ìĬ¹ ĠìķĦ Ġì´Īëĵ±íķ...
                                ...                        
275120    ëįĶ ĠëĤĺìķĦ ê°ĢìĦľ ĠëĤĺì¤ĳìĹĲëĬĶ ĠíķĺëĬĺ ĠëıĦì...
275123               ìĸ´ ĠíĥĿìĭľ ê°Ģ ĠìŀĪìĿĦ Ġê±° Ġê°Ļìķł .
275125    ìļ°ë¦¬ ëĤĺ ëĿ¼ëıĦ ĠìĻĦìłĦ ĠìŀĲìľ¨ì£¼íĸī ì°¨ë¥¼...
275126    ëĮĢ ë°ķ ìĿ´ëĭ¤ ĠìĿ´ìłľ Ġì¹ł ĠëħĦ ĠëĤ¨ìķĺ ëĦ¤ Ġ...
275134    ê·¼ëį° Ġë°ľíĳľë¥¼ Ġíķľëĭ¤ê³ł Ġíķ´ìĦľ ĠìĿ´ê²Į ĠëĶ±
Name: 표준어, Length: 110459, dtype: object

In [16]:
ori_df = pd.read_csv("../datas/output/jeonla_dialect_bpe_integration.csv", index_col=0)

ori_df[ori_df["셋"] == "test"] = df
ori_df[ori_df["셋"] == "test"]

,방언,표준어,연령대,출처 파일,길이,셋
0,ëĤĺëĬĶ ĠíļĮìĤ¬ ĠíķĻêµĲìĿĺ ĠìĿ´ìķ¼ê¸°ë¥¼ Ġíķĺê³...,ëĤĺëĬĶ ĠíļĮìĤ¬ ĠíķĻêµĲìĿĺ ĠìĿ´ìķ¼ê¸°ë¥¼ Ġíķĺê³...,20대,DJDD20000247.json,14,test
2,ëıĻ ì°½ Ġì¹ľêµ¬ëĵ¤ìĿĢ ĠìĿ´ìłľ Ġì´Īëĵ±íķĻêµĲ Ġë...,ëıĻ ì°½ Ġì¹ľêµ¬ëĵ¤ìĿĢ ĠìĿ´ìłľ Ġì´Īëĵ±íķĻêµĲ Ġë...,20대,DJDD20000247.json,7,test
3,ìĿĮ Ġ Ġì¹ľêµ¬ëĵ¤ìĿĢ Ġê±°ìĿĺ ĠìĿ´ìłľ Ġì´Īëĵ±íķĻ...,ìĿĮ Ġ Ġì¹ľêµ¬ëĵ¤ìĿĢ Ġê±°ìĿĺ ĠìĿ´ìłľ Ġì´Īëĵ±íķĻ...,20대,DJDD20000247.json,5,test
4,ìĿ´ëłĩê²Į Ġì¹ľêµ¬ìĿ¸ Ġìķł ĠìŀĪê³ł Ġì¤ĳíķĻêµĲ ì...,ìĿ´ëłĩê²Į Ġì¹ľêµ¬ìĿ¸ Ġìķł ĠìŀĪê³ł Ġì¤ĳíķĻêµĲ ì...,20대,DJDD20000247.json,14,test
5,ì¹ľêµ¬ ëĵ¤ëıĦ ĠìĿ´ìłľ Ġì ´¤ ìĬ¹ ĠìķĦ Ġì´Īëĵ±íķ...,ì¹ľêµ¬ ëĵ¤ëıĦ ĠìĿ´ìłľ Ġì ´¤ ìĬ¹ ĠìķĦ Ġì´Īëĵ±íķ...,20대,DJDD20000247.json,13,test
...,...,...,...,...,...,...
275120,ëįĶ ĠëĤĺìķĦ ê°ĢìĦľ ĠëĤĺì¤ĳìĹĲëĬĶ ĠíķĺëĬĺ ĠëıĦì...,ëįĶ ĠëĤĺìķĦ ê°ĢìĦľ ĠëĤĺì¤ĳìĹĲëĬĶ ĠíķĺëĬĺ ĠëıĦì...,30대,DJSX20003150.json,8,test
275123,ìĸ´ ĠíĥĿìĭľ ê°Ģ ĠìŀĪìĿĦ Ġê±° Ġê°Ļìķł .,ìĸ´ ĠíĥĿìĭľ ê°Ģ ĠìŀĪìĿĦ Ġê±° Ġê°Ļìķł .,30대,DJSX20003150.json,5,test
275125,ìļ°ë¦¬ ëĤĺ ëĿ¼ëıĦ ĠìĻĦìłĦ ĠìŀĲìľ¨ì£¼íĸī ì°¨ë¥¼...,ìļ°ë¦¬ ëĤĺ ëĿ¼ëıĦ ĠìĻĦìłĦ ĠìŀĲìľ¨ì£¼íĸī ì°¨ë¥¼...,30대,DJSX20003150.json,12,test
275126,ëĮĢ ë°ķ ìĿ´ëĭ¤ ĠìĿ´ìłľ Ġì¹ł ĠëħĦ ĠëĤ¨ìķĺ ëĦ¤ Ġ...,ëĮĢ ë°ķ ìĿ´ëĭ¤ ĠìĿ´ìłľ Ġì¹ł ĠëħĦ ĠëĤ¨ìķĺ ëĦ¤ Ġ...,30대,DJSX20003150.json,7,test


In [17]:
ori_df.to_csv("../datas/output/jeonla_dialect_bpe_integration.csv")

### SentencePiece

In [6]:
df = pd.read_csv("../datas/output/jeonla_dialect_SentencePiece_integration.csv", index_col=0)
df

,방언,표준어,연령대,출처 파일,길이,셋
0,▁막상 ▁완전 ▁돈이 ▁급하게 ▁필요한 ▁것도 ▁아니라서 ▁그냥 ▁용돈벌이 ▁정도로...,▁막상 ▁완전 ▁돈이 ▁급하게 ▁필요한 ▁것도 ▁아니라서 ▁그냥 ▁용돈벌이 ▁정도로...,20대,DJSX20002683.json,15,train
1,▁좋은 ▁추억이 ▁있어가지고 ▁다시 ▁키운다 면 ▁진돗개 를 ▁키우고 ▁싶고,▁좋은 ▁추억이 ▁있어가지고 ▁다시 ▁키운다 면 ▁진돗개 를 ▁키우고 ▁싶고,30대,DJDG20002125.json,8,train
2,▁쫌 ▁보면 ▁인제 ▁세계 ▁인제,▁조금 ▁보면 ▁인제 ▁세계 ▁인제,30대,DJSX20004304.json,5,train
3,▁가시나 들 ▁있으니까 ▁뭐 ~ ▁일부러 ▁뭐 ~ ▁쫌 ▁져 주는 ▁척 ▁또 ▁한다...,▁계집아이 들 ▁있으니까 ▁뭐 ~ ▁일부러 ▁뭐 ~ ▁조금 ▁져 주는 ▁척 ▁또 ▁...,20대,DJDG20001833.json,12,train
4,▁거기서 ▁어 ▁저 ▁새끼 ▁왜 ▁미니언 ▁안 ▁먹는데 ▁씨 ▁때려 야지 ▁이러지 ...,▁거기서 ▁어 ▁저 ▁새끼 ▁왜 ▁미니언 ▁안 ▁먹는데 ▁씨 ▁때려 야지 ▁이러지 ...,20대,DJDD20000034.json,12,train
...,...,...,...,...,...,...
275119,▁일반 ▁사람들도 ▁쉽게 ▁쉽게 ▁구매할 ▁수 ▁있을 ▁정도의 ▁기술력 을 ▁갖춰 ...,▁일반 ▁사람들도 ▁쉽게 ▁쉽게 ▁구매할 ▁수 ▁있을 ▁정도의 ▁기술력 을 ▁갖춰 ...,30대,DJSX20003150.json,12,test
275120,▁더 ▁나아가서 ▁나중에는 ▁하늘 ▁도시 ▁날아 다닐 ▁것 ▁같애 .,▁더 ▁나아가서 ▁나중에는 ▁하늘 ▁도시 ▁날아 다닐 ▁것 ▁같애 .,30대,DJSX20003150.json,8,test
275123,▁어 ▁택시 가 ▁있을 ▁거 ▁같애 .,▁어 ▁택시 가 ▁있을 ▁거 ▁같애 .,30대,DJSX20003150.json,5,test
275133,▁이 ▁천 ▁이 ▁십 ▁칠 ▁년이 ▁이 ▁천 ▁이 ▁십 ▁칠 ▁년에,▁이 ▁천 ▁이 ▁십 ▁칠 ▁년이 ▁이 ▁천 ▁이 ▁십 ▁칠 ▁년에,30대,DJSX20003150.json,12,test


In [10]:
df = df[df["셋"] == "test"]
df

,방언,표준어,연령대,출처 파일,길이,셋
0,▁나는 ▁회사 ▁학교 의 ▁이야기를 ▁하고 ▁싶어 ▁형은 ▁있잖아요 ▁친구들 ▁같은...,▁나는 ▁회사 ▁학교 의 ▁이야기를 ▁하고 ▁싶어 ▁형은 ▁있잖아요 ▁친구들 ▁같은...,20대,DJDD20000247.json,14,test
4,▁이렇게 ▁친구인 ▁애 ▁있고 ▁중학교 에도 ▁친구들 ▁있고 ▁한데 ▁근데 ▁형은 ...,▁이렇게 ▁친구인 ▁애 ▁있고 ▁중학교 에도 ▁친구들 ▁있고 ▁한데 ▁근데 ▁형은 ...,20대,DJDD20000247.json,14,test
5,▁친구들도 ▁이제 ▁ 촤 승 ▁아 ▁초등학교 ▁중학교 ▁이런 ▁것보다는 ▁고등학교 ...,▁친구들도 ▁이제 ▁ 촤 승 ▁아 ▁초등학교 ▁중학교 ▁이런 ▁것보다는 ▁고등학교 ...,20대,DJDD20000247.json,13,test
7,▁진짜 ▁친한 ▁애들 ▁고등학교 ▁때 ▁친구들 ▁이렇게 ▁진짜 ▁무리 ▁무리 ▁지어...,▁진짜 ▁친한 ▁애들 ▁고등학교 ▁때 ▁친구들 ▁이렇게 ▁진짜 ▁무리 ▁무리 ▁지어...,20대,DJDD20000247.json,19,test
9,▁또 ▁이제 ▁또 ▁나뉘어 ▁그게 ▁무리 에도 ▁나 뉜 다,▁또 ▁이제 ▁또 ▁나뉘어 ▁그게 ▁무리 에도 ▁나 뉜 다,20대,DJDD20000247.json,7,test
...,...,...,...,...,...,...
275119,▁일반 ▁사람들도 ▁쉽게 ▁쉽게 ▁구매할 ▁수 ▁있을 ▁정도의 ▁기술력 을 ▁갖춰 ...,▁일반 ▁사람들도 ▁쉽게 ▁쉽게 ▁구매할 ▁수 ▁있을 ▁정도의 ▁기술력 을 ▁갖춰 ...,30대,DJSX20003150.json,12,test
275120,▁더 ▁나아가서 ▁나중에는 ▁하늘 ▁도시 ▁날아 다닐 ▁것 ▁같애 .,▁더 ▁나아가서 ▁나중에는 ▁하늘 ▁도시 ▁날아 다닐 ▁것 ▁같애 .,30대,DJSX20003150.json,8,test
275123,▁어 ▁택시 가 ▁있을 ▁거 ▁같애 .,▁어 ▁택시 가 ▁있을 ▁거 ▁같애 .,30대,DJSX20003150.json,5,test
275133,▁이 ▁천 ▁이 ▁십 ▁칠 ▁년이 ▁이 ▁천 ▁이 ▁십 ▁칠 ▁년에,▁이 ▁천 ▁이 ▁십 ▁칠 ▁년이 ▁이 ▁천 ▁이 ▁십 ▁칠 ▁년에,30대,DJSX20003150.json,12,test


In [7]:
import sentencepiece as spm

spm.SentencePieceTrainer.train(input='../datas/output/jeonla_dialect_data_text_sampled.txt', model_prefix='jeonla_sp', vocab_size=30000)

In [9]:
sp_tokenizer = spm.SentencePieceProcessor(model_file='../model_storage/stan-JL_SP/jeonla_test_sp.model')

In [13]:
df.loc[df.index, "방언"] = [sp_tokenizer.decode(sentence.split()) for sentence in df["방언"]]
df.loc[df.index, "표준어"] = [sp_tokenizer.decode(sentence.split()) for sentence in df["표준어"]]

df

,방언,표준어,연령대,출처 파일,길이,셋
0,나는 회사 학교의 이야기를 하고 싶어 형은 있잖아요 친구들 같은 거 다 동창 친구들...,나는 회사 학교의 이야기를 하고 싶어 형은 있잖아요 친구들 같은 거 다 동창 친구들...,20대,DJDD20000247.json,14,test
4,이렇게 친구인 애 있고 중학교에도 친구들 있고 한데 근데 형은 솔직히 말하면 그거 있잖아,이렇게 친구인 애 있고 중학교에도 친구들 있고 한데 근데 형은 솔직히 말하면 그거 있잖아,20대,DJDD20000247.json,14,test
5,친구들도 이제 촤승 아 초등학교 중학교 이런 것보다는 고등학교에서 진짜 친구 나뉜다...,친구들도 이제 촤승 아 초등학교 중학교 이런 것보다는 고등학교에서 진짜 친구 나뉜다...,20대,DJDD20000247.json,13,test
7,진짜 친한 애들 고등학교 때 친구들 이렇게 진짜 무리 무리 지어 다녔던 애들 이렇게...,진짜 친한 애들 고등학교 때 친구들 이렇게 진짜 무리 무리 지어 다녔던 애들 이렇게...,20대,DJDD20000247.json,19,test
9,또 이제 또 나뉘어 그게 무리에도 나뉜다,또 이제 또 나뉘어 그게 무리에도 나뉜다,20대,DJDD20000247.json,7,test
...,...,...,...,...,...,...
275119,일반 사람들도 쉽게 쉽게 구매할 수 있을 정도의 기술력을 갖춰 버렸지 그러면은,일반 사람들도 쉽게 쉽게 구매할 수 있을 정도의 기술력을 갖춰 버렸지 그러면은,30대,DJSX20003150.json,12,test
275120,더 나아가서 나중에는 하늘 도시 날아다닐 것 같애.,더 나아가서 나중에는 하늘 도시 날아다닐 것 같애.,30대,DJSX20003150.json,8,test
275123,어 택시가 있을 거 같애.,어 택시가 있을 거 같애.,30대,DJSX20003150.json,5,test
275133,이 천 이 십 칠 년이 이 천 이 십 칠 년에,이 천 이 십 칠 년이 이 천 이 십 칠 년에,30대,DJSX20003150.json,12,test


In [14]:
sp_tokenizer = spm.SentencePieceProcessor(model_file='../model_storage/stan-JL_SP/jeonla_sp.model')

code_result_dialect = sp_tokenizer.encode(df["방언"].to_list(), out_type=str)
code_result_standard = sp_tokenizer.encode(df["표준어"].to_list(), out_type=str)

In [16]:
df.loc[df.index, "방언"] = [' '.join(res) for res in code_result_dialect]
df["방언"]

C:\Users\wnsgh\AppData\Local\Temp\ipykernel_10584\1695088950.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["방언"] = [' '.join(res) for res in code_result_dialect]


0         ▁나는 ▁회사 ▁학교 의 ▁이야기를 ▁하고 ▁싶어 ▁형은 ▁있잖아요 ▁친구들 ▁같은...
4         ▁이렇게 ▁친구 인 ▁애 ▁있고 ▁중학교 에도 ▁친구들 ▁있고 ▁한데 ▁근데 ▁형은...
5         ▁친구들도 ▁이제 ▁ 촤 승 ▁아 ▁초등학교 ▁중학교 ▁이런 ▁것보다는 ▁고등학교 ...
7         ▁진짜 ▁친한 ▁애들 ▁고등학교 ▁때 ▁친구들 ▁이렇게 ▁진짜 ▁무리 ▁무리 ▁지어...
9                          ▁또 ▁이제 ▁또 ▁나뉘어 ▁그게 ▁무리 에도 ▁나 뉜 다
                                ...                        
275119    ▁일반 ▁사람들도 ▁쉽게 ▁쉽게 ▁구매할 ▁수 ▁있을 ▁정도의 ▁기술 력을 ▁갖춰 ...
275120              ▁더 ▁나아가 서 ▁나중에는 ▁하늘 ▁도시 ▁날아 다닐 ▁것 ▁같애 .
275123                                ▁어 ▁택시 가 ▁있을 ▁거 ▁같애 .
275133                ▁이 ▁천 ▁이 ▁십 ▁칠 ▁년이 ▁이 ▁천 ▁이 ▁십 ▁칠 ▁년에
275134                            ▁근데 ▁발표 를 ▁한다고 ▁해서 ▁이게 ▁딱
Name: 방언, Length: 110459, dtype: object

In [17]:
df.loc[df.index, "표준어"] = [' '.join(res) for res in code_result_standard]
df["표준어"]

C:\Users\wnsgh\AppData\Local\Temp\ipykernel_10584\519260210.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["표준어"] = [' '.join(res) for res in code_result_standard]


0         ▁나는 ▁회사 ▁학교 의 ▁이야기를 ▁하고 ▁싶어 ▁형은 ▁있잖아요 ▁친구들 ▁같은...
4         ▁이렇게 ▁친구 인 ▁애 ▁있고 ▁중학교 에도 ▁친구들 ▁있고 ▁한데 ▁근데 ▁형은...
5         ▁친구들도 ▁이제 ▁ 촤 승 ▁아 ▁초등학교 ▁중학교 ▁이런 ▁것보다는 ▁고등학교 ...
7         ▁진짜 ▁친한 ▁애들 ▁고등학교 ▁때 ▁친구들 ▁이렇게 ▁진짜 ▁무리 ▁무리 ▁지어...
9                          ▁또 ▁이제 ▁또 ▁나뉘어 ▁그게 ▁무리 에도 ▁나 뉜 다
                                ...                        
275119    ▁일반 ▁사람들도 ▁쉽게 ▁쉽게 ▁구매할 ▁수 ▁있을 ▁정도의 ▁기술 력을 ▁갖춰 ...
275120              ▁더 ▁나아가 서 ▁나중에는 ▁하늘 ▁도시 ▁날아 다닐 ▁것 ▁같애 .
275123                                ▁어 ▁택시 가 ▁있을 ▁거 ▁같애 .
275133                ▁이 ▁천 ▁이 ▁십 ▁칠 ▁년이 ▁이 ▁천 ▁이 ▁십 ▁칠 ▁년에
275134                            ▁근데 ▁발표 를 ▁한다고 ▁해서 ▁이게 ▁딱
Name: 표준어, Length: 110459, dtype: object

In [18]:
ori_df = pd.read_csv("../datas/output/jeonla_dialect_SentencePiece_integration.csv", index_col=0)

ori_df[ori_df["셋"] == "test"] = df
ori_df[ori_df["셋"] == "test"]

,방언,표준어,연령대,출처 파일,길이,셋
0,▁나는 ▁회사 ▁학교 의 ▁이야기를 ▁하고 ▁싶어 ▁형은 ▁있잖아요 ▁친구들 ▁같은...,▁나는 ▁회사 ▁학교 의 ▁이야기를 ▁하고 ▁싶어 ▁형은 ▁있잖아요 ▁친구들 ▁같은...,20대,DJDD20000247.json,14,test
4,▁이렇게 ▁친구 인 ▁애 ▁있고 ▁중학교 에도 ▁친구들 ▁있고 ▁한데 ▁근데 ▁형은...,▁이렇게 ▁친구 인 ▁애 ▁있고 ▁중학교 에도 ▁친구들 ▁있고 ▁한데 ▁근데 ▁형은...,20대,DJDD20000247.json,14,test
5,▁친구들도 ▁이제 ▁ 촤 승 ▁아 ▁초등학교 ▁중학교 ▁이런 ▁것보다는 ▁고등학교 ...,▁친구들도 ▁이제 ▁ 촤 승 ▁아 ▁초등학교 ▁중학교 ▁이런 ▁것보다는 ▁고등학교 ...,20대,DJDD20000247.json,13,test
7,▁진짜 ▁친한 ▁애들 ▁고등학교 ▁때 ▁친구들 ▁이렇게 ▁진짜 ▁무리 ▁무리 ▁지어...,▁진짜 ▁친한 ▁애들 ▁고등학교 ▁때 ▁친구들 ▁이렇게 ▁진짜 ▁무리 ▁무리 ▁지어...,20대,DJDD20000247.json,19,test
9,▁또 ▁이제 ▁또 ▁나뉘어 ▁그게 ▁무리 에도 ▁나 뉜 다,▁또 ▁이제 ▁또 ▁나뉘어 ▁그게 ▁무리 에도 ▁나 뉜 다,20대,DJDD20000247.json,7,test
...,...,...,...,...,...,...
275119,▁일반 ▁사람들도 ▁쉽게 ▁쉽게 ▁구매할 ▁수 ▁있을 ▁정도의 ▁기술 력을 ▁갖춰 ...,▁일반 ▁사람들도 ▁쉽게 ▁쉽게 ▁구매할 ▁수 ▁있을 ▁정도의 ▁기술 력을 ▁갖춰 ...,30대,DJSX20003150.json,12,test
275120,▁더 ▁나아가 서 ▁나중에는 ▁하늘 ▁도시 ▁날아 다닐 ▁것 ▁같애 .,▁더 ▁나아가 서 ▁나중에는 ▁하늘 ▁도시 ▁날아 다닐 ▁것 ▁같애 .,30대,DJSX20003150.json,8,test
275123,▁어 ▁택시 가 ▁있을 ▁거 ▁같애 .,▁어 ▁택시 가 ▁있을 ▁거 ▁같애 .,30대,DJSX20003150.json,5,test
275133,▁이 ▁천 ▁이 ▁십 ▁칠 ▁년이 ▁이 ▁천 ▁이 ▁십 ▁칠 ▁년에,▁이 ▁천 ▁이 ▁십 ▁칠 ▁년이 ▁이 ▁천 ▁이 ▁십 ▁칠 ▁년에,30대,DJSX20003150.json,12,test


In [19]:
ori_df.to_csv("../datas/output/jeonla_dialect_SentencePiece_integration.csv")

## 제주 방언

### BPE

In [39]:
df = pd.read_csv("../datas/output/jeju_dialect_bpe_integration.csv", index_col=0)
df

,방언,표준어,연령대,출처 파일,길이,셋
0,ëĤľ Ġì§Ħì§ľ Ġë²Įì´Ī ĠíķłëķĮ Ġë±Ģ ëĤĺìĺ¤ ìŀĸìķĦ...,ëĤľ Ġì§Ħì§ľ Ġë²Įì´Ī ĠíķłëķĮ Ġë±Ģ ëĤĺìĺ¤ ìŀĸìķĦ...,50대,DZES20000002.json,6,train
9,ìķĦ ĠìĺĪ ì´Ī íķĺëĭ¤ê°Ģ ìĿ´ Ġë±Ģ ĠëĤĺìĺ¨ê±°ìķ¼ ...,ìķĦ ĠìĺĪ ì´Ī íķĺëĭ¤ê°Ģ Ġë±Ģ ĠëĤĺìĺ¨ê±°ìķ¼ Ġë±Ģ...,50대,DZES20000002.json,8,train
11,ìķĦ Ġê²ħíķĺëĭĪê¹Į ìĿ´ ĠìĺĪ ì´Ī ĠíķłëķĲ Ġì¡°ìĭ¬...,ìķĦ Ġê·¸ëŁ¬ëĭĪê¹Į ĠìĺĪ ì´Ī ĠíķłëķĲ Ġì¡°ìĭ¬íķ´ì...,50대,DZES20000002.json,6,train
14,ìłĢê¸° ĠëıĮë©©ìĿ´ Ġê°ĻìĿĢ Ġê±° ĠìŀĪìŀĸìķĦ,ìłĢê¸° ĠëıĮë©©ìĿ´ Ġê°ĻìĿĢ Ġê±° ĠìŀĪìŀĸìķĦ,50대,DZES20000002.json,5,train
17,ëĤĺ ĠìķĦëĬĶ ĠìĤ¬ëŀĮìĿ´ Ġë²Į ĠìĿ´ê±°ìĿ´ Ġê·¸ê±°...,ëĤĺ ĠìķĦëĬĶ ĠìĤ¬ëŀĮìĿ´ Ġë²Į ĠìĿ´ê±° Ġê·¸ê±° Ġë...,50대,DZES20000002.json,8,train
...,...,...,...,...,...,...
333778,ìķĦ ĠìĭŃìĿ¼ ĠìĭŃìĿ´ Ġìĭ¬ ìĤ¼ ĠìķĦ ĠìĭŃìĤ¼ ĠìĿ¼,ìķĦ ĠìĭŃìĿ¼ ĠìĭŃìĿ´ Ġìĭ¬ ìĤ¼ ĠìķĦ ĠìĭŃìĤ¼ ĠìĿ¼,30대,DZJD21002409.json,7,test
333780,ìķĦ Ġê²ħ Ġíķ´ Ġê²ħ Ġíķ´ ĠëĤĺ Ġìĸ´ì°¨íĶ¼ ĠìĿ¼ìļ...,ìķĦ Ġê·¸ëłĩê²Į Ġíķ´ Ġê·¸ëłĩê²Į Ġíķ´ ĠëĤĺ Ġìĸ´ì...,30대,DZJD21002409.json,9,test
333784,ê°Ģ ëĭ¤ê°Ģ ĠëŃĲ Ġíķĺë©´ Ġê³µíķŃ ìľ¼ë¡ľ Ġë³´ëŁ¬...,ê°Ģ ëĭ¤ê°Ģ ĠëŃĲ Ġíķĺë©´ Ġê³µíķŃ ìľ¼ë¡ľ Ġë³´ëŁ¬...,30대,DZJD21002409.json,6,test
333797,ë§Ī ë¬´ë¦¬ Ġíķ´ê·¸ëĦ¹ ĠìłĢëħģìĹĲ Ġìŀłê¹Ĳ ĠëĤĺê...,ë§Ī ë¬´ë¦¬ Ġíķ´ìĦľ ĠìłĢëħģìĹĲ Ġìŀłê¹Ĳ ĠëĤĺê°Ī ...,30대,DZJD21002409.json,5,test


In [21]:
from tokenizers import ByteLevelBPETokenizer

bpe_tokenizer = ByteLevelBPETokenizer()
bpe_tokenizer.train(
    files="../datas/output/jeju_dialect_data_text.txt",
)
bpe_tokenizer.save_model("./")

['./vocab.json', './merges.txt']

In [40]:
df = df[df["셋"] == "test"]
df

,방언,표준어,연령대,출처 파일,길이,셋
1,ìĸ´ Ġê·¸ê±° ĠëĮĢíĻĶ Ġì£¼ìłľëĬĶ Ġìĺ¤ëĬĺ ĠëĮĢíĻĶ...,ìĸ´ Ġê·¸ê±° ĠëĮĢíĻĶ Ġì£¼ìłľëĬĶ Ġìĺ¤ëĬĺ ĠëĮĢíĻĶ...,50대,DZES20000001.json,7,test
2,ìĸ´ ĠëªħìłĪ ĠìĦ¤ ĠëªħìłĪ Ġì¶ĶìĦĿ ĠëªħìłĪ ĠìļĶë...,ìĸ´ ĠëªħìłĪ ĠìĦ¤ ĠëªħìłĪ Ġì¶ĶìĦĿ ĠëªħìłĪ ĠìļĶë...,50대,DZES20000001.json,10,test
28,ê·¸ëŀĺê°Ģì§Ģê³ł Ġê·¸ê±°ë¥¼ Ġë»£ë»£ íķĺê²Į Ġë§Ĳ...,ê·¸ëŀĺê°Ģì§Ģê³ł Ġê·¸ê±°ë¥¼ Ġë»£ë»£ íķĺê²Į Ġë§Ĳ...,50대,DZES20000001.json,8,test
34,ê³¼ ì¦Ĳ ìĿĦ ĠíķĺëĬĶëį° Ġê·¸ê±°ë¥¼ Ġìľ¼ Ġë°Ģìĸ´...,ê³¼ ì¦Ĳ ìĿĦ ĠíķĺëĬĶëį° Ġê·¸ê±°ë¥¼ Ġìľ¼ Ġë°Ģ ìĸ...,50대,DZES20000001.json,8,test
36,ê·¸ëŁ¬ë©´ìĿĢ Ġê·¸ê±°ë¥¼ ĠìĿ´ìłľ Ġê¸°ë¦ĦìĹĲ ĠíĬ...,ê·¸ëŁ¬ë©´ìĿĢ Ġê·¸ê±°ë¥¼ ĠìĿ´ìłľ Ġê¸°ë¦ĦìĹĲ ĠíĬ...,50대,DZES20000001.json,10,test
...,...,...,...,...,...,...
333778,ìķĦ ĠìĭŃìĿ¼ ĠìĭŃìĿ´ Ġìĭ¬ ìĤ¼ ĠìķĦ ĠìĭŃìĤ¼ ĠìĿ¼,ìķĦ ĠìĭŃìĿ¼ ĠìĭŃìĿ´ Ġìĭ¬ ìĤ¼ ĠìķĦ ĠìĭŃìĤ¼ ĠìĿ¼,30대,DZJD21002409.json,7,test
333780,ìķĦ Ġê²ħ Ġíķ´ Ġê²ħ Ġíķ´ ĠëĤĺ Ġìĸ´ì°¨íĶ¼ ĠìĿ¼ìļ...,ìķĦ Ġê·¸ëłĩê²Į Ġíķ´ Ġê·¸ëłĩê²Į Ġíķ´ ĠëĤĺ Ġìĸ´ì...,30대,DZJD21002409.json,9,test
333784,ê°Ģ ëĭ¤ê°Ģ ĠëŃĲ Ġíķĺë©´ Ġê³µíķŃ ìľ¼ë¡ľ Ġë³´ëŁ¬...,ê°Ģ ëĭ¤ê°Ģ ĠëŃĲ Ġíķĺë©´ Ġê³µíķŃ ìľ¼ë¡ľ Ġë³´ëŁ¬...,30대,DZJD21002409.json,6,test
333797,ë§Ī ë¬´ë¦¬ Ġíķ´ê·¸ëĦ¹ ĠìłĢëħģìĹĲ Ġìŀłê¹Ĳ ĠëĤĺê...,ë§Ī ë¬´ë¦¬ Ġíķ´ìĦľ ĠìłĢëħģìĹĲ Ġìŀłê¹Ĳ ĠëĤĺê°Ī ...,30대,DZJD21002409.json,5,test


In [45]:
bpe_tokenizer.from_file("../model_storage/stan-JJ_bpe/test_vocab.json", "../model_storage/stan-JJ_bpe/test_merges.txt")

for i in range(len(df)):
    is_unk = False
    dia = df.iloc[i, 0]
    token_ids = [bpe_tokenizer.token_to_id(token) for token in dia.split()]
    df.iloc[i, 0] = bpe_tokenizer.decode([token for token in token_ids if token is not None])
    
    stan = df.iloc[i, 1]
    token_ids = [bpe_tokenizer.token_to_id(token) for token in stan.split()]
    df.iloc[i, 1] = bpe_tokenizer.decode([token for token in token_ids if token is not None])

print(df)

                                         방언  \
1                     어 그거 대화 주제는 오늘 대화 주제는   
2            어 명절 설 명절 추석 명절 요로케 나누어서 해볼 거예   
28      그래가지고 그거를 뻣뻣하게 말르면은 그거를 기름에 튀긴단 말이야   
34           과즐을 하는데 그거를 으 밀어그내 인제 바짝 마르잖아?   
36          그러면은 그거를 이제 기름에 튀겨 그럼 얘가 오르잖아 이   
...                                     ...   
333778                    아 십일 십이 심삼 아 십삼 일   
333780             아 겅 해 겅 해 나 어차피 일요일날 감시난   
333784                 가다가 뭐 하면 공항으로 보러 가게.   
333797                   마 해그넹 저녁에 잠깐 나가크라.   
333800                      난 차 어시난 우리 집 못가   

                                        표준어  연령대              출처 파일  길이     셋  
1                     어 그거 대화 주제는 오늘 대화 주제는  50대  DZES20000001.json   7  test  
2           어 명절 설 명절 추석 명절 요로케 나누어서 해볼 거예요  50대  DZES20000001.json  10  test  
28      그래가지고 그거를 뻣뻣하게 말르면은 그거를 기름에 튀긴단 말이야  50대  DZES20000001.json   8  test  
34            과즐을 하는데 그거를 으 밀어서 인제 바짝 마르잖아?  50대  DZES20000001.json   8  test  
36         그러면은 그거를 이제 기름에 튀겨 그럼 얘가

In [46]:
bpe_tokenizer.from_file("./vocab.json", "./merges.txt")

Tokenizer(vocabulary_size=30000, model=ByteLevelBPE, add_prefix_space=False, lowercase=False, dropout=None, unicode_normalizer=None, continuing_subword_prefix=None, end_of_word_suffix=None, trim_offsets=False)

In [47]:
code_result_dialect = bpe_tokenizer.encode_batch(df["방언"])
code_result_standard = bpe_tokenizer.encode_batch(df["표준어"])

In [48]:
df.loc[df.index, "방언"] = [' '.join(res.tokens) for res in code_result_dialect]
df["방언"]

1         ìĸ´ Ġê·¸ê±° ĠëĮĢíĻĶ Ġì£¼ìłľëĬĶ Ġìĺ¤ëĬĺ ĠëĮĢíĻĶ...
2         ìĸ´ ĠëªħìłĪ ĠìĦ¤ ĠëªħìłĪ Ġì¶ĶìĦĿ ĠëªħìłĪ ĠìļĶë...
28        ê·¸ëŀĺê°Ģì§Ģê³ł Ġê·¸ê±°ë¥¼ Ġë»£ë»£ íķĺê²Į Ġë§Ĳ...
34        ê³¼ ì¦Ĳ ìĿĦ ĠíķĺëĬĶëį° Ġê·¸ê±°ë¥¼ Ġìľ¼ Ġë°Ģìĸ´...
36        ê·¸ëŁ¬ë©´ìĿĢ Ġê·¸ê±°ë¥¼ ĠìĿ´ìłľ Ġê¸°ë¦ĦìĹĲ ĠíĬ...
                                ...                        
333778       ìķĦ ĠìĭŃìĿ¼ ĠìĭŃìĿ´ Ġìĭ¬ ìĤ¼ ĠìķĦ ĠìĭŃìĤ¼ ĠìĿ¼
333780    ìķĦ Ġê²ħ Ġíķ´ Ġê²ħ Ġíķ´ ĠëĤĺ Ġìĸ´ì°¨íĶ¼ ĠìĿ¼ìļ...
333784    ê°Ģëĭ¤ê°Ģ ĠëŃĲ Ġíķĺë©´ Ġê³µíķŃ ìľ¼ë¡ľ Ġë³´ëŁ¬ ...
333797    ë§Ī Ġíķ´ê·¸ëĦ¹ ĠìłĢëħģìĹĲ Ġìŀłê¹Ĳ ĠëĤĺê°Ģ íģ¬ë...
333800             ëĤľ Ġì°¨ Ġìĸ´ìĭľëĤľ Ġìļ°ë¦¬ Ġì§ĳ Ġëª»ê°Ģ
Name: 방언, Length: 80061, dtype: object

In [49]:
df.loc[df.index, "표준어"] = [' '.join(res.tokens) for res in code_result_standard]
df["표준어"]

1         ìĸ´ Ġê·¸ê±° ĠëĮĢíĻĶ Ġì£¼ìłľëĬĶ Ġìĺ¤ëĬĺ ĠëĮĢíĻĶ...
2         ìĸ´ ĠëªħìłĪ ĠìĦ¤ ĠëªħìłĪ Ġì¶ĶìĦĿ ĠëªħìłĪ ĠìļĶë...
28        ê·¸ëŀĺê°Ģì§Ģê³ł Ġê·¸ê±°ë¥¼ Ġë»£ë»£ íķĺê²Į Ġë§Ĳ...
34        ê³¼ ì¦Ĳ ìĿĦ ĠíķĺëĬĶëį° Ġê·¸ê±°ë¥¼ Ġìľ¼ Ġë°Ģ ìĸ...
36        ê·¸ëŁ¬ë©´ìĿĢ Ġê·¸ê±°ë¥¼ ĠìĿ´ìłľ Ġê¸°ë¦ĦìĹĲ ĠíĬ...
                                ...                        
333778       ìķĦ ĠìĭŃìĿ¼ ĠìĭŃìĿ´ Ġìĭ¬ ìĤ¼ ĠìķĦ ĠìĭŃìĤ¼ ĠìĿ¼
333780    ìķĦ Ġê·¸ëłĩê²Į Ġíķ´ Ġê·¸ëłĩê²Į Ġíķ´ ĠëĤĺ Ġìĸ´ì...
333784    ê°Ģëĭ¤ê°Ģ ĠëŃĲ Ġíķĺë©´ Ġê³µíķŃ ìľ¼ë¡ľ Ġë³´ëŁ¬ ...
333797         ë§Ī Ġíķ´ìĦľ ĠìłĢëħģìĹĲ Ġìŀłê¹Ĳ ĠëĤĺê°Ī ê²Į .
333800          ëĤľ Ġì°¨ ĠìĹĨìľ¼ëĭĪê¹Į Ġìļ°ë¦¬ Ġì§ĳ Ġëª»ê°Ģ
Name: 표준어, Length: 80061, dtype: object

In [50]:
ori_df = pd.read_csv("../datas/output/jeju_dialect_bpe_integration.csv", index_col=0)

ori_df[ori_df["셋"] == "test"] = df
ori_df[ori_df["셋"] == "test"]

,방언,표준어,연령대,출처 파일,길이,셋
1,ìĸ´ Ġê·¸ê±° ĠëĮĢíĻĶ Ġì£¼ìłľëĬĶ Ġìĺ¤ëĬĺ ĠëĮĢíĻĶ...,ìĸ´ Ġê·¸ê±° ĠëĮĢíĻĶ Ġì£¼ìłľëĬĶ Ġìĺ¤ëĬĺ ĠëĮĢíĻĶ...,50대,DZES20000001.json,7,test
2,ìĸ´ ĠëªħìłĪ ĠìĦ¤ ĠëªħìłĪ Ġì¶ĶìĦĿ ĠëªħìłĪ ĠìļĶë...,ìĸ´ ĠëªħìłĪ ĠìĦ¤ ĠëªħìłĪ Ġì¶ĶìĦĿ ĠëªħìłĪ ĠìļĶë...,50대,DZES20000001.json,10,test
28,ê·¸ëŀĺê°Ģì§Ģê³ł Ġê·¸ê±°ë¥¼ Ġë»£ë»£ íķĺê²Į Ġë§Ĳ...,ê·¸ëŀĺê°Ģì§Ģê³ł Ġê·¸ê±°ë¥¼ Ġë»£ë»£ íķĺê²Į Ġë§Ĳ...,50대,DZES20000001.json,8,test
34,ê³¼ ì¦Ĳ ìĿĦ ĠíķĺëĬĶëį° Ġê·¸ê±°ë¥¼ Ġìľ¼ Ġë°Ģìĸ´...,ê³¼ ì¦Ĳ ìĿĦ ĠíķĺëĬĶëį° Ġê·¸ê±°ë¥¼ Ġìľ¼ Ġë°Ģ ìĸ...,50대,DZES20000001.json,8,test
36,ê·¸ëŁ¬ë©´ìĿĢ Ġê·¸ê±°ë¥¼ ĠìĿ´ìłľ Ġê¸°ë¦ĦìĹĲ ĠíĬ...,ê·¸ëŁ¬ë©´ìĿĢ Ġê·¸ê±°ë¥¼ ĠìĿ´ìłľ Ġê¸°ë¦ĦìĹĲ ĠíĬ...,50대,DZES20000001.json,10,test
...,...,...,...,...,...,...
333778,ìķĦ ĠìĭŃìĿ¼ ĠìĭŃìĿ´ Ġìĭ¬ ìĤ¼ ĠìķĦ ĠìĭŃìĤ¼ ĠìĿ¼,ìķĦ ĠìĭŃìĿ¼ ĠìĭŃìĿ´ Ġìĭ¬ ìĤ¼ ĠìķĦ ĠìĭŃìĤ¼ ĠìĿ¼,30대,DZJD21002409.json,7,test
333780,ìķĦ Ġê²ħ Ġíķ´ Ġê²ħ Ġíķ´ ĠëĤĺ Ġìĸ´ì°¨íĶ¼ ĠìĿ¼ìļ...,ìķĦ Ġê·¸ëłĩê²Į Ġíķ´ Ġê·¸ëłĩê²Į Ġíķ´ ĠëĤĺ Ġìĸ´ì...,30대,DZJD21002409.json,9,test
333784,ê°Ģëĭ¤ê°Ģ ĠëŃĲ Ġíķĺë©´ Ġê³µíķŃ ìľ¼ë¡ľ Ġë³´ëŁ¬ ...,ê°Ģëĭ¤ê°Ģ ĠëŃĲ Ġíķĺë©´ Ġê³µíķŃ ìľ¼ë¡ľ Ġë³´ëŁ¬ ...,30대,DZJD21002409.json,6,test
333797,ë§Ī Ġíķ´ê·¸ëĦ¹ ĠìłĢëħģìĹĲ Ġìŀłê¹Ĳ ĠëĤĺê°Ģ íģ¬ë...,ë§Ī Ġíķ´ìĦľ ĠìłĢëħģìĹĲ Ġìŀłê¹Ĳ ĠëĤĺê°Ī ê²Į .,30대,DZJD21002409.json,5,test


In [51]:
ori_df.to_csv("../datas/output/jeju_dialect_bpe_integration.csv")

### SentencePiece

In [52]:
df = pd.read_csv("../datas/output/jeju_dialect_SentencePiece_integration.csv", index_col=0)
df

,방언,표준어,연령대,출처 파일,길이,셋
0,▁난 ▁진짜 ▁벌초 ▁할때 ▁뱀 나 오 잖아 ▁뱀 도,▁난 ▁진짜 ▁벌초 ▁할때 ▁뱀 나 오 잖아 ▁뱀 도,50대,DZES20000002.json,6,train
9,▁아 ▁예 초 하다가 이 ▁뱀 ▁나온거야 ▁뱀이 ▁하나 ▁도 ▁아니고,▁아 ▁예 초 하다가 ▁뱀 ▁나온거야 ▁뱀이 ▁하나 ▁도 ▁아니고,50대,DZES20000002.json,8,train
11,▁아 ▁겅하니까 이 ▁예 초 ▁할땐 ▁조심해야 ▁돼,▁아 ▁그러니까 ▁예 초 ▁할땐 ▁조심해야 ▁돼,50대,DZES20000002.json,6,train
14,▁저기 ▁돌멩이 ▁같은 ▁거 ▁있잖아,▁저기 ▁돌멩이 ▁같은 ▁거 ▁있잖아,50대,DZES20000002.json,5,train
17,▁나 ▁아는 ▁사람이 ▁벌 ▁이거 이 ▁그거 ▁벌초 하다가 ▁예 초 기가,▁나 ▁아는 ▁사람이 ▁벌 ▁이거 ▁그거 ▁벌초 하다가 ▁예 초 기가,50대,DZES20000002.json,8,train
...,...,...,...,...,...,...
333774,▁아 ▁너 ▁너 ▁십삼 ▁일,▁아 ▁너 ▁너 ▁십삼 ▁일,30대,DZJD21002409.json,5,test
333778,▁아 ▁십일 ▁십이 ▁심 삼 ▁아 ▁십삼 ▁일,▁아 ▁십일 ▁십이 ▁심 삼 ▁아 ▁십삼 ▁일,30대,DZJD21002409.json,7,test
333782,▁일요일 ▁오후 ▁두 ▁시 ▁비행기,▁일요일 ▁오후 ▁두 ▁시 ▁비행기,30대,DZJD21002409.json,5,test
333790,▁막걸리 나 ▁한 ▁잔 ▁바 ▁땡기 던가,▁막걸리 나 ▁한 ▁잔 ▁바 ▁땡기 던가,30대,DZJD21002409.json,5,test


In [53]:
df = df[df["셋"] == "test"]
df

,방언,표준어,연령대,출처 파일,길이,셋
0,▁언니 ▁만낭 ▁반 가 워 ▁아 ▁오늘 ▁제주 ▁방언 ▁에이 ▁아이 ▁데이터,▁언니 ▁만나서 ▁반 가 워 ▁아 ▁오늘 ▁제주 ▁방언 ▁에이 ▁아이 ▁데이터,50대,DZES20000001.json,10,test
2,▁어 ▁명절 ▁설 ▁명절 ▁추석 ▁명절 ▁요 로케 ▁나누어 서 ▁해볼 ▁거 예,▁어 ▁명절 ▁설 ▁명절 ▁추석 ▁명절 ▁요 로케 ▁나누어 서 ▁해볼 ▁거예요,50대,DZES20000001.json,10,test
7,▁어 ▁그믄 ▁언니네 ▁설 ▁명절 ▁때 ▁음식 ▁어떵 ▁해 ?,▁어 ▁그러면 ▁언니네 ▁설 ▁명절 ▁때 ▁음식 ▁어떻게 ▁해 ?,50대,DZES20000001.json,9,test
11,▁음식을 ▁막 ▁준비 했던 ▁기억이 ▁나 게,▁음식을 ▁막 ▁준비 했던 ▁기억이 ▁나,50대,DZES20000001.json,5,test
15,▁게 ▁나 ▁어린 ▁워낙 ▁어려서,▁# 게 ▁나 ▁어린 ▁워낙 ▁어려서,50대,DZES20000001.json,5,test
...,...,...,...,...,...,...
333774,▁아 ▁너 ▁너 ▁십삼 ▁일,▁아 ▁너 ▁너 ▁십삼 ▁일,30대,DZJD21002409.json,5,test
333778,▁아 ▁십일 ▁십이 ▁심 삼 ▁아 ▁십삼 ▁일,▁아 ▁십일 ▁십이 ▁심 삼 ▁아 ▁십삼 ▁일,30대,DZJD21002409.json,7,test
333782,▁일요일 ▁오후 ▁두 ▁시 ▁비행기,▁일요일 ▁오후 ▁두 ▁시 ▁비행기,30대,DZJD21002409.json,5,test
333790,▁막걸리 나 ▁한 ▁잔 ▁바 ▁땡기 던가,▁막걸리 나 ▁한 ▁잔 ▁바 ▁땡기 던가,30대,DZJD21002409.json,5,test


In [ ]:
import sentencepiece as spm

spm.SentencePieceTrainer.train(input='../datas/output/jeju_dialect_data_text.txt', model_prefix='jeonla_sp', vocab_size=30000)

In [56]:
sp_tokenizer = spm.SentencePieceProcessor(model_file='../model_storage/stan-JJ_SP/jeju_sp.model')

In [57]:
df.loc[df.index, "방언"] = [sp_tokenizer.decode(sentence.split()) for sentence in df["방언"]]
df.loc[df.index, "표준어"] = [sp_tokenizer.decode(sentence.split()) for sentence in df["표준어"]]

df

,방언,표준어,연령대,출처 파일,길이,셋
0,언니 만낭 반가워 아 오늘 제주 방언 에이 아이 데이터,언니 만나서 반가워 아 오늘 제주 방언 에이 아이 데이터,50대,DZES20000001.json,10,test
2,어 명절 설 명절 추석 명절 요로케 나누어서 해볼 거예,어 명절 설 명절 추석 명절 요로케 나누어서 해볼 거예요,50대,DZES20000001.json,10,test
7,어 그믄 언니네 설 명절 때 음식 어떵 해?,어 그러면 언니네 설 명절 때 음식 어떻게 해?,50대,DZES20000001.json,9,test
11,음식을 막 준비했던 기억이 나게,음식을 막 준비했던 기억이 나,50대,DZES20000001.json,5,test
15,게 나 어린 워낙 어려서,#게 나 어린 워낙 어려서,50대,DZES20000001.json,5,test
...,...,...,...,...,...,...
333774,아 너 너 십삼 일,아 너 너 십삼 일,30대,DZJD21002409.json,5,test
333778,아 십일 십이 심삼 아 십삼 일,아 십일 십이 심삼 아 십삼 일,30대,DZJD21002409.json,7,test
333782,일요일 오후 두 시 비행기,일요일 오후 두 시 비행기,30대,DZJD21002409.json,5,test
333790,막걸리나 한 잔 바 땡기던가,막걸리나 한 잔 바 땡기던가,30대,DZJD21002409.json,5,test


In [58]:
sp_tokenizer = spm.SentencePieceProcessor(model_file='../model_storage/stan-JJ_SP/jeju_data_sp.model')

code_result_dialect = sp_tokenizer.encode(df["방언"].to_list(), out_type=str)
code_result_standard = sp_tokenizer.encode(df["표준어"].to_list(), out_type=str)

In [59]:
df.loc[df.index, "방언"] = [' '.join(res) for res in code_result_dialect]
df["방언"]

C:\Users\wnsgh\AppData\Local\Temp\ipykernel_10584\1695088950.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["방언"] = [' '.join(res) for res in code_result_dialect]


0           ▁언니 ▁만낭 ▁반가워 ▁아 ▁오늘 ▁제주 ▁방언 ▁에이 ▁아이 ▁데이터
2         ▁어 ▁명절 ▁설 ▁명절 ▁추석 ▁명절 ▁요로케 ▁나누어 서 ▁해볼 ▁거 예
7                 ▁어 ▁그믄 ▁언니네 ▁설 ▁명절 ▁때 ▁음식 ▁어떵 ▁해 ?
11                          ▁음식을 ▁막 ▁준비 했던 ▁기억이 ▁나 게
15                                ▁게 ▁나 ▁어린 ▁워낙 ▁어려서
                             ...                    
333774                               ▁아 ▁너 ▁너 ▁십삼 ▁일
333778                     ▁아 ▁십일 ▁십이 ▁심 삼 ▁아 ▁십삼 ▁일
333782                           ▁일요일 ▁오후 ▁두 ▁시 ▁비행기
333790                        ▁막걸리 나 ▁한 ▁잔 ▁바 ▁땡기 던가
333800                         ▁난 ▁차 ▁어시난 ▁우리 ▁집 ▁못가
Name: 방언, Length: 80061, dtype: object

In [60]:
df.loc[df.index, "표준어"] = [' '.join(res) for res in code_result_standard]
df["표준어"]

0          ▁언니 ▁만나서 ▁반가워 ▁아 ▁오늘 ▁제주 ▁방언 ▁에이 ▁아이 ▁데이터
2         ▁어 ▁명절 ▁설 ▁명절 ▁추석 ▁명절 ▁요로케 ▁나누어 서 ▁해볼 ▁거예요
7               ▁어 ▁그러면 ▁언니네 ▁설 ▁명절 ▁때 ▁음식 ▁어떻게 ▁해 ?
11                            ▁음식을 ▁막 ▁준비 했던 ▁기억이 ▁나
15                              ▁# 게 ▁나 ▁어린 ▁워낙 ▁어려서
                             ...                    
333774                               ▁아 ▁너 ▁너 ▁십삼 ▁일
333778                     ▁아 ▁십일 ▁십이 ▁심 삼 ▁아 ▁십삼 ▁일
333782                           ▁일요일 ▁오후 ▁두 ▁시 ▁비행기
333790                        ▁막걸리 나 ▁한 ▁잔 ▁바 ▁땡기 던가
333800                        ▁난 ▁차 ▁없으니까 ▁우리 ▁집 ▁못가
Name: 표준어, Length: 80061, dtype: object

In [61]:
ori_df = pd.read_csv("../datas/output/jeju_dialect_SentencePiece_integration.csv", index_col=0)

ori_df[ori_df["셋"] == "test"] = df
ori_df[ori_df["셋"] == "test"]

,방언,표준어,연령대,출처 파일,길이,셋
0,▁언니 ▁만낭 ▁반가워 ▁아 ▁오늘 ▁제주 ▁방언 ▁에이 ▁아이 ▁데이터,▁언니 ▁만나서 ▁반가워 ▁아 ▁오늘 ▁제주 ▁방언 ▁에이 ▁아이 ▁데이터,50대,DZES20000001.json,10,test
2,▁어 ▁명절 ▁설 ▁명절 ▁추석 ▁명절 ▁요로케 ▁나누어 서 ▁해볼 ▁거 예,▁어 ▁명절 ▁설 ▁명절 ▁추석 ▁명절 ▁요로케 ▁나누어 서 ▁해볼 ▁거예요,50대,DZES20000001.json,10,test
7,▁어 ▁그믄 ▁언니네 ▁설 ▁명절 ▁때 ▁음식 ▁어떵 ▁해 ?,▁어 ▁그러면 ▁언니네 ▁설 ▁명절 ▁때 ▁음식 ▁어떻게 ▁해 ?,50대,DZES20000001.json,9,test
11,▁음식을 ▁막 ▁준비 했던 ▁기억이 ▁나 게,▁음식을 ▁막 ▁준비 했던 ▁기억이 ▁나,50대,DZES20000001.json,5,test
15,▁게 ▁나 ▁어린 ▁워낙 ▁어려서,▁# 게 ▁나 ▁어린 ▁워낙 ▁어려서,50대,DZES20000001.json,5,test
...,...,...,...,...,...,...
333774,▁아 ▁너 ▁너 ▁십삼 ▁일,▁아 ▁너 ▁너 ▁십삼 ▁일,30대,DZJD21002409.json,5,test
333778,▁아 ▁십일 ▁십이 ▁심 삼 ▁아 ▁십삼 ▁일,▁아 ▁십일 ▁십이 ▁심 삼 ▁아 ▁십삼 ▁일,30대,DZJD21002409.json,7,test
333782,▁일요일 ▁오후 ▁두 ▁시 ▁비행기,▁일요일 ▁오후 ▁두 ▁시 ▁비행기,30대,DZJD21002409.json,5,test
333790,▁막걸리 나 ▁한 ▁잔 ▁바 ▁땡기 던가,▁막걸리 나 ▁한 ▁잔 ▁바 ▁땡기 던가,30대,DZJD21002409.json,5,test


In [62]:
ori_df.to_csv("../datas/output/jeju_dialect_SentencePiece_integration.csv")

## 강원 방언

### BPE

In [64]:
df = pd.read_csv("../datas/output/gangwon_dialect_bpe_integration.csv", index_col=0)
df

,방언,표준어,연령대,출처 파일,길이,셋
6,ë°Ķëĭ¤ ìĿĺ Ġë¬´ìĬ¨ Ġìĸĺê¸°ë¥¼ Ġíķ´ìķ¼ ĠëĲĺëĤĺ ?,ë°Ķëĭ¤ ìĿĺ Ġë¬´ìĬ¨ Ġìĸĺê¸°ë¥¼ Ġíķ´ìķ¼ ĠëĲĺëĤĺ ?,60대 이상,DGDQ20000020.json,5,train
10,ìĺĽëĤłìĹĲ Ġë°Ķëĭ¤ìĹĲìĦľ ĠëĤ´ê°Ģ Ġìŀĺíķľëĭ¤ê³ł ...,ìĺĽëĤłìĹĲ Ġë°Ķëĭ¤ìĹĲìĦľ ĠëĤ´ê°Ģ Ġìŀĺíķľëĭ¤ê³ł ...,60대 이상,DGDQ20000020.json,8,train
13,êµ¬ìĭŃ Ġëªĩ ĠëħĦ ĠëıĦ ìĹĲ ĠíĸĪëĬĶëį°ìļĶ ?,êµ¬ìĭŃ Ġëªĩ ĠëħĦ ĠëıĦ ìĹĲ ĠíĸĪëĬĶëį°ìļĶ ?,60대 이상,DGDQ20000020.json,5,train
14,ê·¸ëķĮê°Ģ ĠìĹłë¹Ħ ìĶ¨ Ġë°©ìĨ¡ êµŃìĹĲìĦľ Ġíķ´ëħ...,ê·¸ëķĮê°Ģ ĠìĹłë¹Ħ ìĶ¨ Ġë°©ìĨ¡ êµŃìĹĲìĦľ Ġíķ´ëħ...,60대 이상,DGDQ20000020.json,9,train
15,ê·¸ëŀĺìĦľ ĠìĹłë¹Ħ ìĶ¨ Ġë°©ìĨ¡ Ġê³µê°ľ ĠíĻĢ Ġê°...,ê·¸ëŀĺìĦľ ĠìĹłë¹Ħ ìĶ¨ Ġë°©ìĨ¡ Ġê³µê°ľ ĠíĻĢ Ġê°...,60대 이상,DGDQ20000020.json,10,train
...,...,...,...,...,...,...
276792,ê·¼ëį° Ġì§ĪìĿ´ Ġêµīìŀ¥íŀĪ ĠëĤ®ìĿĢ ĠìĨĮê´´ ê¸° ...,ê·¼ëį° Ġì§ĪìĿ´ Ġêµīìŀ¥íŀĪ ĠëĤ®ìĿĢ Ġê³łê¸° ìĹ¬ ...,30대,DGIN21801133.json,6,test
276793,ìĤ¬ìĭ¤ ĠëŃĲ ~ Ġìĸ´ ~ Ġê¸°ìĸµìĿĢ ĠìķĪ ĠëĤĺê³ł Ġ...,ìĤ¬ìĭ¤ ĠëŃĲ ~ Ġìĸ´ ~ Ġê¸°ìĸµìĿĢ ĠìķĪ ĠëĤĺê³ł Ġ...,30대,DGIN21801133.json,13,test
276795,ìĨĮ ê´´ ê¸°ëĬĶ ĠìĤ¬ìĭ¤ Ġìĸ´ ~ ĠëĤĺìĿ´ê°Ģ Ġëĵ¤ê...,ìĨĮ ê³łê¸°ëĬĶ ĠìĤ¬ìĭ¤ Ġìĸ´ ~ ĠëĤĺìĿ´ê°Ģ Ġëĵ¤ê³...,30대,DGIN21801133.json,6,test
276796,ë¨¹ ìĹĪìĹĪëįĺ Ġê²ĥ Ġê°Ļìķł ĠìĤ¬ìĭ¤ Ġìĸ´ ~ ĠìłĬ...,ë¨¹ ìĹĪìĹĪëįĺ Ġê²ĥ Ġê°ĻìķĦ ĠìĤ¬ìĭ¤ Ġìĸ´ ~ ĠìłĬ...,30대,DGIN21801133.json,15,test


In [63]:
from tokenizers import ByteLevelBPETokenizer

bpe_tokenizer = ByteLevelBPETokenizer()
bpe_tokenizer.train(
    files="../datas/output/gangwon_dialect_data_text.txt",
)
bpe_tokenizer.save_model("./")

['./vocab.json', './merges.txt']

In [65]:
df = df[df["셋"] == "test"]
df

,방언,표준어,연령대,출처 파일,길이,셋
0,ëĤĺëĬĶ ĠìĤ¬ìĭ¤ ĠëĤ´ ĠìĿ¸ìĥĿ ĠìŀĲì²´ê°Ģ Ġì¦Ŀë§Ĳ...,ëĤĺëĬĶ ĠìĤ¬ìĭ¤ ĠëĤ´ ĠìĿ¸ìĥĿ ĠìŀĲì²´ê°Ģ Ġìłķë§Ĳ...,20대,DGIN21500168.json,11,test
2,ê·¸ëŁ°ëį° ĠìĿ´ìłľ ĠëĤĺëĬĶ Ġë¬´ì¡°ê·¼ Ġê¸°ë¶Ħ Ġ...,ê·¸ëŁ°ëį° ĠìĿ´ìłľ ĠëĤĺëĬĶ Ġë¬´ì¡°ê±´ Ġê¸°ë¶Ħ Ġ...,20대,DGIN21500168.json,12,test
3,ê¸°ë¶Ħ Ġì¢ĭìĿĢ Ġëĭ¤ìĿ´ìĸ´íĬ¸ë¥¼ Ġíķĺê³ł Ġìĭ¶ìĿ...,ê¸°ë¶Ħ Ġì¢ĭìĿĢ Ġëĭ¤ìĿ´ìĸ´íĬ¸ë¥¼ Ġíķĺê³ł Ġìĭ¶ìĿ...,20대,DGIN21500168.json,5,test
11,ìĸ´ê±°ì§Ģë¡ľ Ġíķĺì§Ģ ĠìķĬìľ¼ëł¤ê³ł Ġëħ¸ëł¥ìĿĦ ...,ìĸ´ê±°ì§Ģë¡ľ Ġíķĺì§Ģ ĠìķĬìľ¼ëł¤ê³ł Ġëħ¸ëł¥ìĿĦ ...,20대,DGIN21500168.json,6,test
14,ìĿ¼ëĭ¨ìĿĢ ĠëĤĺëĬĶ Ġëĭ¤ìĿ´ìĸ´íĬ¸ë¥¼ Ġíķł ĠëķĮ Ġ...,ìĿ¼ëĭ¨ìĿĢ ĠëĤĺëĬĶ Ġëĭ¤ìĿ´ìĸ´íĬ¸ë¥¼ Ġíķł ĠëķĮ Ġ...,20대,DGIN21500168.json,8,test
...,...,...,...,...,...,...
276792,ê·¼ëį° Ġì§ĪìĿ´ Ġêµīìŀ¥íŀĪ ĠëĤ®ìĿĢ ĠìĨĮê´´ ê¸° ...,ê·¼ëį° Ġì§ĪìĿ´ Ġêµīìŀ¥íŀĪ ĠëĤ®ìĿĢ Ġê³łê¸° ìĹ¬ ...,30대,DGIN21801133.json,6,test
276793,ìĤ¬ìĭ¤ ĠëŃĲ ~ Ġìĸ´ ~ Ġê¸°ìĸµìĿĢ ĠìķĪ ĠëĤĺê³ł Ġ...,ìĤ¬ìĭ¤ ĠëŃĲ ~ Ġìĸ´ ~ Ġê¸°ìĸµìĿĢ ĠìķĪ ĠëĤĺê³ł Ġ...,30대,DGIN21801133.json,13,test
276795,ìĨĮ ê´´ ê¸°ëĬĶ ĠìĤ¬ìĭ¤ Ġìĸ´ ~ ĠëĤĺìĿ´ê°Ģ Ġëĵ¤ê...,ìĨĮ ê³łê¸°ëĬĶ ĠìĤ¬ìĭ¤ Ġìĸ´ ~ ĠëĤĺìĿ´ê°Ģ Ġëĵ¤ê³...,30대,DGIN21801133.json,6,test
276796,ë¨¹ ìĹĪìĹĪëįĺ Ġê²ĥ Ġê°Ļìķł ĠìĤ¬ìĭ¤ Ġìĸ´ ~ ĠìłĬ...,ë¨¹ ìĹĪìĹĪëįĺ Ġê²ĥ Ġê°ĻìķĦ ĠìĤ¬ìĭ¤ Ġìĸ´ ~ ĠìłĬ...,30대,DGIN21801133.json,15,test


In [67]:
bpe_tokenizer.from_file("../model_storage/stan-GW_bpe/test_vocab.json", "../model_storage/stan-GW_bpe/test_merges.txt")

for i in range(len(df)):
    is_unk = False
    dia = df.iloc[i, 0]
    token_ids = [bpe_tokenizer.token_to_id(token) for token in dia.split()]
    df.iloc[i, 0] = bpe_tokenizer.decode([token for token in token_ids if token is not None])
    
    stan = df.iloc[i, 1]
    token_ids = [bpe_tokenizer.token_to_id(token) for token in stan.split()]
    df.iloc[i, 1] = bpe_tokenizer.decode([token for token in token_ids if token is not None])

print(df)

                                                  방언  \
0                 나는 사실 내 인생 자체가 증말 다이어트라고 해도 과언이 아닐   
2        그런데 이제 나는 무조근 기분 나쁜 다이어트가 아니라 쪼끔 내 몸이 건강해지는   
3                                 기분 좋은 다이어트를 하고 싶은데   
11                                하지 않으려고 노력을 하고 있어.   
14                           일단은 나는 다이어트를 할 때 칠십 퍼센트   
...                                              ...   
276792                             근데 질이 굉장히 낮은기여 갖고   
276793  사실 뭐~ 어~ 기억은 안 나고 그때는 난 돼지고기를 먹기 위해서 많이 갔었지.   
276795                          소괴기는 사실 어~ 나이가 들고 어~   
276796   먹 것 같애 사실 어~ 젊었을 때나 어렸을 때라던가 이럴 때는 워낙기가 보니까   
276797       내가 일을 하면서 뭔가 쉽사리 접근할 수 있었던 이제 부분은 아니었어서   

                                                 표준어  연령대              출처 파일  \
0                 나는 사실 내 인생 자체가 정말 다이어트라고 해도 과언이 아닐  20대  DGIN21500168.json   
2        그런데 이제 나는 무조건 기분 나쁜 다이어트가 아니라 쪼끔 내 몸이 건강해지는  20대  DGIN21500168.json   
3                                 기분 좋은 다이어트를 하고 싶은데  20대  DGIN21500168.json   
11     

In [68]:
bpe_tokenizer.from_file("./vocab.json", "./merges.txt")

Tokenizer(vocabulary_size=30000, model=ByteLevelBPE, add_prefix_space=False, lowercase=False, dropout=None, unicode_normalizer=None, continuing_subword_prefix=None, end_of_word_suffix=None, trim_offsets=False)

In [69]:
code_result_dialect = bpe_tokenizer.encode_batch(df["방언"])
code_result_standard = bpe_tokenizer.encode_batch(df["표준어"])

In [70]:
df.loc[df.index, "방언"] = [' '.join(res.tokens) for res in code_result_dialect]
df["방언"]

0         ëĤĺëĬĶ ĠìĤ¬ìĭ¤ ĠëĤ´ ĠìĿ¸ìĥĿ ĠìŀĲì²´ê°Ģ Ġì¦Ŀë§Ĳ...
2         ê·¸ëŁ°ëį° ĠìĿ´ìłľ ĠëĤĺëĬĶ Ġë¬´ì¡°ê·¼ Ġê¸°ë¶Ħ Ġ...
3         ê¸°ë¶Ħ Ġì¢ĭìĿĢ Ġëĭ¤ìĿ´ìĸ´íĬ¸ë¥¼ Ġíķĺê³ł Ġìĭ¶ìĿ...
11        Ġíķĺì§Ģ ĠìķĬìľ¼ëł¤ê³ł Ġëħ¸ëł¥ìĿĦ Ġíķĺê³ł ĠìŀĪì...
14        ìĿ¼ëĭ¨ìĿĢ ĠëĤĺëĬĶ Ġëĭ¤ìĿ´ìĸ´íĬ¸ë¥¼ Ġíķł ĠëķĮ Ġ...
                                ...                        
276792    ê·¼ëį° Ġì§ĪìĿ´ Ġêµīìŀ¥íŀĪ ĠëĤ®ìĿĢ ê¸° ìĹ¬ Ġê°ĸê³ł
276793    ìĤ¬ìĭ¤ ĠëŃĲ ~ Ġìĸ´ ~ Ġê¸°ìĸµìĿĢ ĠìķĪ ĠëĤĺê³ł Ġ...
276795    ìĨĮ ê´´ ê¸°ëĬĶ ĠìĤ¬ìĭ¤ Ġìĸ´ ~ ĠëĤĺìĿ´ê°Ģ Ġëĵ¤ê...
276796    ë¨¹ Ġê²ĥ Ġê°Ļìķł ĠìĤ¬ìĭ¤ Ġìĸ´ ~ ĠìłĬìĹĪìĿĦ Ġëķ...
276797    ëĤ´ê°Ģ ĠìĿ¼ìĿĦ Ġíķĺë©´ìĦľ ĠëŃĶê°Ģ Ġìī½ ìĤ¬ë¦¬ ...
Name: 방언, Length: 91345, dtype: object

In [71]:
df.loc[df.index, "표준어"] = [' '.join(res.tokens) for res in code_result_standard]
df["표준어"]

0         ëĤĺëĬĶ ĠìĤ¬ìĭ¤ ĠëĤ´ ĠìĿ¸ìĥĿ ĠìŀĲì²´ê°Ģ Ġìłķë§Ĳ...
2         ê·¸ëŁ°ëį° ĠìĿ´ìłľ ĠëĤĺëĬĶ Ġë¬´ì¡°ê±´ Ġê¸°ë¶Ħ Ġ...
3         ê¸°ë¶Ħ Ġì¢ĭìĿĢ Ġëĭ¤ìĿ´ìĸ´íĬ¸ë¥¼ Ġíķĺê³ł Ġìĭ¶ìĿ...
11        Ġíķĺì§Ģ ĠìķĬìľ¼ëł¤ê³ł Ġëħ¸ëł¥ìĿĦ Ġíķĺê³ł ĠìŀĪì...
14        ìĿ¼ëĭ¨ìĿĢ ĠëĤĺëĬĶ Ġëĭ¤ìĿ´ìĸ´íĬ¸ë¥¼ Ġíķł ĠëķĮ Ġ...
                                ...                        
276792    ê·¼ëį° Ġì§ĪìĿ´ Ġêµīìŀ¥íŀĪ ĠëĤ®ìĿĢ Ġê³łê¸° ìĹ¬ ...
276793    ìĤ¬ìĭ¤ ĠëŃĲ ~ Ġìĸ´ ~ Ġê¸°ìĸµìĿĢ ĠìķĪ ĠëĤĺê³ł Ġ...
276795    ìĨĮ ê³łê¸°ëĬĶ ĠìĤ¬ìĭ¤ Ġìĸ´ ~ ĠëĤĺìĿ´ê°Ģ Ġëĵ¤ê³...
276796    ë¨¹ Ġê²ĥ Ġê°ĻìķĦ ĠìĤ¬ìĭ¤ Ġìĸ´ ~ ĠìłĬìĹĪìĿĦ Ġëķ...
276797    ëĤ´ê°Ģ ĠìĿ¼ìĿĦ Ġíķĺë©´ìĦľ ĠëŃĶê°Ģ Ġìī½ ìĤ¬ë¦¬ ...
Name: 표준어, Length: 91345, dtype: object

In [72]:
ori_df = pd.read_csv("../datas/output/gangwon_dialect_bpe_integration.csv", index_col=0)

ori_df[ori_df["셋"] == "test"] = df
ori_df[ori_df["셋"] == "test"]

,방언,표준어,연령대,출처 파일,길이,셋
0,ëĤĺëĬĶ ĠìĤ¬ìĭ¤ ĠëĤ´ ĠìĿ¸ìĥĿ ĠìŀĲì²´ê°Ģ Ġì¦Ŀë§Ĳ...,ëĤĺëĬĶ ĠìĤ¬ìĭ¤ ĠëĤ´ ĠìĿ¸ìĥĿ ĠìŀĲì²´ê°Ģ Ġìłķë§Ĳ...,20대,DGIN21500168.json,11,test
2,ê·¸ëŁ°ëį° ĠìĿ´ìłľ ĠëĤĺëĬĶ Ġë¬´ì¡°ê·¼ Ġê¸°ë¶Ħ Ġ...,ê·¸ëŁ°ëį° ĠìĿ´ìłľ ĠëĤĺëĬĶ Ġë¬´ì¡°ê±´ Ġê¸°ë¶Ħ Ġ...,20대,DGIN21500168.json,12,test
3,ê¸°ë¶Ħ Ġì¢ĭìĿĢ Ġëĭ¤ìĿ´ìĸ´íĬ¸ë¥¼ Ġíķĺê³ł Ġìĭ¶ìĿ...,ê¸°ë¶Ħ Ġì¢ĭìĿĢ Ġëĭ¤ìĿ´ìĸ´íĬ¸ë¥¼ Ġíķĺê³ł Ġìĭ¶ìĿ...,20대,DGIN21500168.json,5,test
11,Ġíķĺì§Ģ ĠìķĬìľ¼ëł¤ê³ł Ġëħ¸ëł¥ìĿĦ Ġíķĺê³ł ĠìŀĪì...,Ġíķĺì§Ģ ĠìķĬìľ¼ëł¤ê³ł Ġëħ¸ëł¥ìĿĦ Ġíķĺê³ł ĠìŀĪì...,20대,DGIN21500168.json,6,test
14,ìĿ¼ëĭ¨ìĿĢ ĠëĤĺëĬĶ Ġëĭ¤ìĿ´ìĸ´íĬ¸ë¥¼ Ġíķł ĠëķĮ Ġ...,ìĿ¼ëĭ¨ìĿĢ ĠëĤĺëĬĶ Ġëĭ¤ìĿ´ìĸ´íĬ¸ë¥¼ Ġíķł ĠëķĮ Ġ...,20대,DGIN21500168.json,8,test
...,...,...,...,...,...,...
276792,ê·¼ëį° Ġì§ĪìĿ´ Ġêµīìŀ¥íŀĪ ĠëĤ®ìĿĢ ê¸° ìĹ¬ Ġê°ĸê³ł,ê·¼ëį° Ġì§ĪìĿ´ Ġêµīìŀ¥íŀĪ ĠëĤ®ìĿĢ Ġê³łê¸° ìĹ¬ ...,30대,DGIN21801133.json,6,test
276793,ìĤ¬ìĭ¤ ĠëŃĲ ~ Ġìĸ´ ~ Ġê¸°ìĸµìĿĢ ĠìķĪ ĠëĤĺê³ł Ġ...,ìĤ¬ìĭ¤ ĠëŃĲ ~ Ġìĸ´ ~ Ġê¸°ìĸµìĿĢ ĠìķĪ ĠëĤĺê³ł Ġ...,30대,DGIN21801133.json,13,test
276795,ìĨĮ ê´´ ê¸°ëĬĶ ĠìĤ¬ìĭ¤ Ġìĸ´ ~ ĠëĤĺìĿ´ê°Ģ Ġëĵ¤ê...,ìĨĮ ê³łê¸°ëĬĶ ĠìĤ¬ìĭ¤ Ġìĸ´ ~ ĠëĤĺìĿ´ê°Ģ Ġëĵ¤ê³...,30대,DGIN21801133.json,6,test
276796,ë¨¹ Ġê²ĥ Ġê°Ļìķł ĠìĤ¬ìĭ¤ Ġìĸ´ ~ ĠìłĬìĹĪìĿĦ Ġëķ...,ë¨¹ Ġê²ĥ Ġê°ĻìķĦ ĠìĤ¬ìĭ¤ Ġìĸ´ ~ ĠìłĬìĹĪìĿĦ Ġëķ...,30대,DGIN21801133.json,15,test


In [73]:
ori_df.to_csv("../datas/output/gangwon_dialect_bpe_integration.csv")

### SentencePiece

In [101]:
df = pd.read_csv("../datas/output/gangwon_dialect_SentencePiece_integration.csv", index_col=0)
df

,방언,표준어,연령대,출처 파일,길이,셋
6,▁바다 의 ▁무슨 ▁얘기를 ▁해야 ▁되나 ?,▁바다 의 ▁무슨 ▁얘기를 ▁해야 ▁되나 ?,60대 이상,DGDQ20000020.json,5,train
10,▁옛날에 ▁바다에서 ▁내가 ▁잘한다고 ▁엠 비 씨 까지 ▁갔다 ▁왔거든요 ▁제가 .,▁옛날에 ▁바다에서 ▁내가 ▁잘한다고 ▁엠 비 씨 까지 ▁갔다 ▁왔거든요 ▁제가 .,60대 이상,DGDQ20000020.json,8,train
13,▁구십 ▁몇 ▁년 ▁ 도 에 ▁했는데요 ?,▁구십 ▁몇 ▁년 ▁ 도 에 ▁했는데요 ?,60대 이상,DGDQ20000020.json,5,train
14,▁그때가 ▁엠 비 씨 ▁방송국 에서 ▁해녀 ▁하라고 ▁또 ▁인터뷰 ▁하자 ▁하대 ?,▁그때가 ▁엠 비 씨 ▁방송국 에서 ▁해녀 ▁하라고 ▁또 ▁인터뷰 ▁하자 ▁하더라 ?,60대 이상,DGDQ20000020.json,9,train
15,▁그래서 ▁엠 비 씨 ▁방송 ▁공개 ▁홀 ▁강릉 ▁가 ▁가지고 ▁노래 도 ▁부르 구,▁그래서 ▁엠 비 씨 ▁방송 ▁공개 ▁홀 ▁강릉 ▁가 ▁가지고 ▁노래 도 ▁부르 구,60대 이상,DGDQ20000020.json,10,train
...,...,...,...,...,...,...
276791,▁인제 ▁괴기 ▁부패 를 ▁많이 ▁갔는데 ▁거기에 도 ▁아마 ▁소고기가 ▁있기는 ▁...,▁인제 ▁고기 ▁부패 를 ▁많이 ▁갔는데 ▁거기에 도 ▁아마 ▁소고기가 ▁있기는 ▁...,30대,DGIN21801133.json,10,test
276792,▁근데 ▁질 이 ▁굉장히 ▁낮은 ▁소괴기 여 ▁갖고,▁근데 ▁질 이 ▁굉장히 ▁낮은 ▁고기 여 ▁갖고,30대,DGIN21801133.json,6,test
276794,▁어렸을 ▁때 ▁삼겹살 이라던가 ▁거기 ▁항상 ▁냉동 이긴 ▁했는데 ▁어 ~ ▁그거...,▁어렸을 ▁때 ▁삼겹살 이라던가 ▁거기 ▁항상 ▁냉동 이긴 ▁했는데 ▁어 ~ ▁그거...,30대,DGIN21801133.json,13,test
276796,▁먹었었던 ▁것 ▁같애 ▁사실 ▁어 ~ ▁젊었 을 ▁때나 ▁어렸을 ▁때 라던가 ▁이...,▁먹었었던 ▁것 ▁같아 ▁사실 ▁어 ~ ▁젊었 을 ▁때나 ▁어렸을 ▁때 라던가 ▁이...,30대,DGIN21801133.json,15,test


In [102]:
df = df[df["셋"] == "test"]
df

,방언,표준어,연령대,출처 파일,길이,셋
2,▁그런데 ▁이제 ▁나는 ▁무조근 ▁기분 ▁나쁜 ▁다이어트가 ▁아니라 ▁쪼끔 ▁내 ▁...,▁그런데 ▁이제 ▁나는 ▁무조건 ▁기분 ▁나쁜 ▁다이어트가 ▁아니라 ▁쪼끔 ▁내 ▁...,20대,DGIN21500168.json,12,test
3,▁기분 ▁좋은 ▁다이어트를 ▁하고 ▁싶은데,▁기분 ▁좋은 ▁다이어트를 ▁하고 ▁싶은데,20대,DGIN21500168.json,5,test
6,▁뭐 ~ ▁어떤 ▁다이어트가 ▁너한테 ▁도움이 ▁됐는지,▁뭐 ~ ▁어떤 ▁다이어트가 ▁너한테 ▁도움이 ▁됐는지,20대,DGIN21500168.json,6,test
8,▁나는 ▁일단 ▁다이어트 ▁한다는 ▁거 ▁자체가 ▁너무,▁나는 ▁일단 ▁다이어트 ▁한다는 ▁거 ▁자체가 ▁너무,20대,DGIN21500168.json,7,test
11,▁어거지로 ▁하지 ▁않으려 고 ▁노력을 ▁하고 ▁있어 .,▁어거지로 ▁하지 ▁않으려 고 ▁노력을 ▁하고 ▁있어 .,20대,DGIN21500168.json,6,test
...,...,...,...,...,...,...
276791,▁인제 ▁괴기 ▁부패 를 ▁많이 ▁갔는데 ▁거기에 도 ▁아마 ▁소고기가 ▁있기는 ▁...,▁인제 ▁고기 ▁부패 를 ▁많이 ▁갔는데 ▁거기에 도 ▁아마 ▁소고기가 ▁있기는 ▁...,30대,DGIN21801133.json,10,test
276792,▁근데 ▁질 이 ▁굉장히 ▁낮은 ▁소괴기 여 ▁갖고,▁근데 ▁질 이 ▁굉장히 ▁낮은 ▁고기 여 ▁갖고,30대,DGIN21801133.json,6,test
276794,▁어렸을 ▁때 ▁삼겹살 이라던가 ▁거기 ▁항상 ▁냉동 이긴 ▁했는데 ▁어 ~ ▁그거...,▁어렸을 ▁때 ▁삼겹살 이라던가 ▁거기 ▁항상 ▁냉동 이긴 ▁했는데 ▁어 ~ ▁그거...,30대,DGIN21801133.json,13,test
276796,▁먹었었던 ▁것 ▁같애 ▁사실 ▁어 ~ ▁젊었 을 ▁때나 ▁어렸을 ▁때 라던가 ▁이...,▁먹었었던 ▁것 ▁같아 ▁사실 ▁어 ~ ▁젊었 을 ▁때나 ▁어렸을 ▁때 라던가 ▁이...,30대,DGIN21801133.json,15,test


In [ ]:
import sentencepiece as spm

spm.SentencePieceTrainer.train(input='../datas/output/jeju_dialect_data_text.txt', model_prefix='jeonla_sp', vocab_size=30000)

KeyboardInterrupt: 

In [103]:
sp_tokenizer = spm.SentencePieceProcessor(model_file='../model_storage/stan-GW_SP/gangwon_sp_test.model')

In [104]:
df.loc[df.index, "방언"] = [sp_tokenizer.decode(sentence.split()) for sentence in df["방언"]]
df.loc[df.index, "표준어"] = [sp_tokenizer.decode(sentence.split()) for sentence in df["표준어"]]

df

,방언,표준어,연령대,출처 파일,길이,셋
2,그런데 이제 나는 무조근 기분 나쁜 다이어트가 아니라 쪼끔 내 몸이 건강해지는,그런데 이제 나는 무조건 기분 나쁜 다이어트가 아니라 쪼끔 내 몸이 건강해지는,20대,DGIN21500168.json,12,test
3,기분 좋은 다이어트를 하고 싶은데,기분 좋은 다이어트를 하고 싶은데,20대,DGIN21500168.json,5,test
6,뭐~ 어떤 다이어트가 너한테 도움이 됐는지,뭐~ 어떤 다이어트가 너한테 도움이 됐는지,20대,DGIN21500168.json,6,test
8,나는 일단 다이어트 한다는 거 자체가 너무,나는 일단 다이어트 한다는 거 자체가 너무,20대,DGIN21500168.json,7,test
11,어거지로 하지 않으려고 노력을 하고 있어.,어거지로 하지 않으려고 노력을 하고 있어.,20대,DGIN21500168.json,6,test
...,...,...,...,...,...,...
276791,인제 괴기 부패를 많이 갔는데 거기에도 아마 소고기가 있기는 했어.,인제 고기 부패를 많이 갔는데 거기에도 아마 소고기가 있기는 했어.,30대,DGIN21801133.json,10,test
276792,근데 질이 굉장히 낮은 소괴기여 갖고,근데 질이 굉장히 낮은 고기여 갖고,30대,DGIN21801133.json,6,test
276794,어렸을 때 삼겹살이라던가 거기 항상 냉동이긴 했는데 어~ 그거를 많이 먹었었던 것 같애.,어렸을 때 삼겹살이라던가 거기 항상 냉동이긴 했는데 어~ 그거를 많이 먹었었던 것 같아.,30대,DGIN21801133.json,13,test
276796,먹었었던 것 같애 사실 어~ 젊었을 때나 어렸을 때라던가 이럴 때는 워낙 소괴기가 ...,먹었었던 것 같아 사실 어~ 젊었을 때나 어렸을 때라던가 이럴 때는 워낙 소고기가 ...,30대,DGIN21801133.json,15,test


In [105]:
sp_tokenizer = spm.SentencePieceProcessor(model_file='../model_storage/stan-GW_SP/gangwon_sp.model')

code_result_dialect = sp_tokenizer.encode(df["방언"].to_list(), out_type=str)
code_result_standard = sp_tokenizer.encode(df["표준어"].to_list(), out_type=str)

In [106]:
df.loc[df.index, "방언"] = [' '.join(res) for res in code_result_dialect]
df["방언"]

2         ▁그런데 ▁이제 ▁나는 ▁무조근 ▁기분 ▁나쁜 ▁다이어트가 ▁아니라 ▁쪼끔 ▁내 ▁...
3                                   ▁기분 ▁좋은 ▁다이어트를 ▁하고 ▁싶은데
6                            ▁뭐 ~ ▁어떤 ▁다이어트가 ▁너한테 ▁도움이 ▁됐는지
8                            ▁나는 ▁일단 ▁다이어트 ▁한다는 ▁거 ▁자체가 ▁너무
11                           ▁어거지로 ▁하지 ▁않으려고 ▁노력을 ▁하고 ▁있어 .
                                ...                        
276791    ▁인제 ▁괴기 ▁부패 를 ▁많이 ▁갔는데 ▁거기에 도 ▁아마 ▁소고기 가 ▁있기는 ...
276792                        ▁근데 ▁질이 ▁굉장히 ▁낮은 ▁소 괴 기 여 ▁갖고
276794    ▁어렸을 ▁때 ▁삼겹살 이라던가 ▁거기 ▁항상 ▁냉동 이긴 ▁했는데 ▁어 ~ ▁그거...
276796    ▁먹었었던 ▁것 ▁같애 ▁사실 ▁어 ~ ▁젊었을 ▁때나 ▁어렸을 ▁때 라던가 ▁이럴...
276797    ▁내가 ▁일을 ▁하면서 ▁뭔가 ▁쉽 사리 ▁접근 할 ▁수 ▁있었던 ▁이제 ▁부분은 ...
Name: 방언, Length: 91345, dtype: object

In [107]:
df.loc[df.index, "표준어"] = [' '.join(res) for res in code_result_standard]
df["표준어"]

2         ▁그런데 ▁이제 ▁나는 ▁무조건 ▁기분 ▁나쁜 ▁다이어트가 ▁아니라 ▁쪼끔 ▁내 ▁...
3                                   ▁기분 ▁좋은 ▁다이어트를 ▁하고 ▁싶은데
6                            ▁뭐 ~ ▁어떤 ▁다이어트가 ▁너한테 ▁도움이 ▁됐는지
8                            ▁나는 ▁일단 ▁다이어트 ▁한다는 ▁거 ▁자체가 ▁너무
11                           ▁어거지로 ▁하지 ▁않으려고 ▁노력을 ▁하고 ▁있어 .
                                ...                        
276791    ▁인제 ▁고기 ▁부패 를 ▁많이 ▁갔는데 ▁거기에 도 ▁아마 ▁소고기 가 ▁있기는 ...
276792                           ▁근데 ▁질이 ▁굉장히 ▁낮은 ▁고기 여 ▁갖고
276794    ▁어렸을 ▁때 ▁삼겹살 이라던가 ▁거기 ▁항상 ▁냉동 이긴 ▁했는데 ▁어 ~ ▁그거...
276796    ▁먹었었던 ▁것 ▁같아 ▁사실 ▁어 ~ ▁젊었을 ▁때나 ▁어렸을 ▁때 라던가 ▁이럴...
276797    ▁내가 ▁일을 ▁하면서 ▁뭔가 ▁쉽 사리 ▁접근 할 ▁수 ▁있었던 ▁이제 ▁부분은 ...
Name: 표준어, Length: 91345, dtype: object

In [108]:
ori_df = pd.read_csv("../datas/output/gangwon_dialect_SentencePiece_integration.csv", index_col=0)

ori_df[ori_df["셋"] == "test"] = df
ori_df[ori_df["셋"] == "test"]

,방언,표준어,연령대,출처 파일,길이,셋
2,▁그런데 ▁이제 ▁나는 ▁무조근 ▁기분 ▁나쁜 ▁다이어트가 ▁아니라 ▁쪼끔 ▁내 ▁...,▁그런데 ▁이제 ▁나는 ▁무조건 ▁기분 ▁나쁜 ▁다이어트가 ▁아니라 ▁쪼끔 ▁내 ▁...,20대,DGIN21500168.json,12,test
3,▁기분 ▁좋은 ▁다이어트를 ▁하고 ▁싶은데,▁기분 ▁좋은 ▁다이어트를 ▁하고 ▁싶은데,20대,DGIN21500168.json,5,test
6,▁뭐 ~ ▁어떤 ▁다이어트가 ▁너한테 ▁도움이 ▁됐는지,▁뭐 ~ ▁어떤 ▁다이어트가 ▁너한테 ▁도움이 ▁됐는지,20대,DGIN21500168.json,6,test
8,▁나는 ▁일단 ▁다이어트 ▁한다는 ▁거 ▁자체가 ▁너무,▁나는 ▁일단 ▁다이어트 ▁한다는 ▁거 ▁자체가 ▁너무,20대,DGIN21500168.json,7,test
11,▁어거지로 ▁하지 ▁않으려고 ▁노력을 ▁하고 ▁있어 .,▁어거지로 ▁하지 ▁않으려고 ▁노력을 ▁하고 ▁있어 .,20대,DGIN21500168.json,6,test
...,...,...,...,...,...,...
276791,▁인제 ▁괴기 ▁부패 를 ▁많이 ▁갔는데 ▁거기에 도 ▁아마 ▁소고기 가 ▁있기는 ...,▁인제 ▁고기 ▁부패 를 ▁많이 ▁갔는데 ▁거기에 도 ▁아마 ▁소고기 가 ▁있기는 ...,30대,DGIN21801133.json,10,test
276792,▁근데 ▁질이 ▁굉장히 ▁낮은 ▁소 괴 기 여 ▁갖고,▁근데 ▁질이 ▁굉장히 ▁낮은 ▁고기 여 ▁갖고,30대,DGIN21801133.json,6,test
276794,▁어렸을 ▁때 ▁삼겹살 이라던가 ▁거기 ▁항상 ▁냉동 이긴 ▁했는데 ▁어 ~ ▁그거...,▁어렸을 ▁때 ▁삼겹살 이라던가 ▁거기 ▁항상 ▁냉동 이긴 ▁했는데 ▁어 ~ ▁그거...,30대,DGIN21801133.json,13,test
276796,▁먹었었던 ▁것 ▁같애 ▁사실 ▁어 ~ ▁젊었을 ▁때나 ▁어렸을 ▁때 라던가 ▁이럴...,▁먹었었던 ▁것 ▁같아 ▁사실 ▁어 ~ ▁젊었을 ▁때나 ▁어렸을 ▁때 라던가 ▁이럴...,30대,DGIN21801133.json,15,test


In [109]:
ori_df.to_csv("../datas/output/gangwon_dialect_SentencePiece_integration.csv")

## 충청 방언

### BPE

In [91]:
df = pd.read_csv("../datas/output/chungcheong_dialect_bpe_integration.csv", index_col=0)
df

,방언,표준어,연령대,출처 파일,길이,셋
0,ìĺģíĻĶ ë³´ëĭ¤ëĬĶ Ġê·¸ëŁ° Ġìª½ìľ¼ë¡ľ Ġì«Į Ġë§İì...,ìĺģíĻĶ ë³´ëĭ¤ëĬĶ Ġê·¸ëŁ° Ġìª½ìľ¼ë¡ľ Ġì¡°ê¸Ī Ġë...,50대,DCNA20001462.json,6,train
1,ìĹĲ ~ ĠìłĢëĬĶ ĠìĽĲëŀĺ Ġê³łëĵ±íķĻìĥĿ ĠëķĮê¹Įì§Ģ,ìĹĲ ~ ĠìłĢëĬĶ ĠìĽĲëŀĺ Ġê³łëĵ±íķĻìĥĿ ĠëķĮê¹Įì§Ģ,20대,DCDG20000795.json,5,train
2,ê·¸ Ġcompanyname ĠíĶĦë¡ľê·¸ëŀ¨ ĠìĿ´ê±¸ Ġë§İìĿ´...,ê·¸ Ġcompanyname ĠíĶĦë¡ľê·¸ëŀ¨ ĠìĿ´ê±¸ Ġë§İìĿ´...,20대,DCDG20001601.json,6,train
3,ê³ł ìľĦíĹĺ Ġê·¸ ~ Ġêµ° ìĿ´ ĠìķĦëĭĪìķ¼ Ġê·¼ëį° ...,ê³ł ìľĦíĹĺ Ġê·¸ ~ Ġêµ° ìĿ´ ĠìķĦëĭĪìķ¼ Ġê·¼ëį° ...,40대,DCEX20001058.json,21,train
4,ì½Ķë¡ľëĤĺ ĠëķĮë¬¸ìĹĲ ĠìĤ¬ëŀĮëĵ¤ìĿ´ Ġë§İìĿ´ Ġì§...,ì½Ķë¡ľëĤĺ ĠëķĮë¬¸ìĹĲ ĠìĤ¬ëŀĮëĵ¤ìĿ´ Ġë§İìĿ´ Ġì§...,40대,DCNA20001390.json,9,train
...,...,...,...,...,...,...
227993,ê·¸ëŁ¬ë©´ìĦľ ĠìĿ¸ìłľ ĠìĿ´ëŁ° Ġìĸĺê¸°ëıĦ ĠìŀĪìŀ...,ê·¸ëŁ¬ë©´ìĦľ ĠìĿ¸ìłľ ĠìĿ´ëŁ° Ġìĸĺê¸°ëıĦ ĠìŀĪìŀ...,50대,DCNA20000036.json,5,test
227995,ê·¸ëŁ° Ġë§ĲìĿ´ ĠìĻĢëĭ¿ ìķĦìļĶ ĠìĿ¸ìłľ ĠëĤĺìĿ´ê...,ê·¸ëŁ° Ġë§ĲìĿ´ ĠìĻĢëĭ¿ ìķĦìļĶ ĠìĿ¸ìłľ ĠëĤĺìĿ´ê...,50대,DCNA20000036.json,7,test
227999,ìĿ¸ìłľ Ġì²ł ëĵ¤ ìŀĲ Ġìŀī ~ ĠìłĢê¸° Ġíķľëĭ¤ê³ł ...,ìĿ¸ìłľ Ġì²ł ëĵ¤ ìŀĲ Ġìŀī ~ ĠìłĢê¸° Ġíķľëĭ¤ê³ł ...,50대,DCNA20000036.json,9,test
228001,ìĸ´ëĸ»ê²Į Ġë³´ë©´ ĠìĦłìĥĿëĭĺ Ġì§Ģê¸Ī Ġê·¸ê²Į Ġ...,ìĸ´ëĸ»ê²Į Ġë³´ë©´ ĠìĦłìĥĿëĭĺ Ġì§Ģê¸Ī Ġê·¸ê²Į Ġ...,50대,DCNA20000036.json,11,test


In [92]:
from tokenizers import ByteLevelBPETokenizer

bpe_tokenizer = ByteLevelBPETokenizer()
bpe_tokenizer.train(
    files="../datas/output/chungcheong_dialect_data_text.txt",
)
bpe_tokenizer.save_model("./")

['./vocab.json', './merges.txt']

In [93]:
df = df[df["셋"] == "test"]
df

,방언,표준어,연령대,출처 파일,길이,셋
0,ìļ°ë¦¬ ì§ĳ Ġê±°ë¶ģìĿ´ Ġìļ©ë´īìĿ´ê°Ģ Ġì§Ģê¸Ī Ġì...,ìļ°ë¦¬ ì§ĳ Ġê±°ë¶ģìĿ´ Ġìļ©ë´īìĿ´ê°Ģ Ġì§Ģê¸Ī Ġì...,50대,DCDG20000009.json,9,test
2,ìĸ´ ~ ĠìķĦíĮĮìĦľ Ġê³ł ë¹Ħê°Ģ ĠìŀĪìĹĪëĬĶëį° Ġì§...,ìĸ´ ~ ĠìķĦíĮĮìĦľ Ġê³ł ë¹Ħê°Ģ ĠìŀĪìĹĪëĬĶëį° Ġì§...,50대,DCDG20000009.json,17,test
4,ê·¸ëŀĺëıĦ Ġìļ©ë´īìĿ´ê°Ģ Ġê°Ģì¡± ĠìĿ¼ ìĽĲ ìľ¼ë¡...,ê·¸ëŀĺëıĦ Ġìļ©ë´īìĿ´ê°Ģ Ġê°Ģì¡± ĠìĿ¼ ìĽĲ ìľ¼ë¡...,50대,DCDG20000009.json,5,test
5,ìķĦ ~ ĠìĺĪìģĺ ê¸´ Ġíķľëį° ĠìĹĦë§ĪëĬĶ ĠëĦĪë¬´ Ġ...,ìķĦ ~ ĠìĺĪìģĺ ê¸´ Ġíķľëį° ĠìĹĦë§ĪëĬĶ ĠëĦĪë¬´ Ġ...,20대,DCDG20000009.json,15,test
9,ê·¸ëĥ¥ Ġê°ĻìĿ´ ĠìĤ¬ëĬĶ ĠëıĻë¬¼ ìĿ´ëĿ¼ëĬĶ Ġê°ľë...,ê·¸ëĥ¥ Ġê°ĻìĿ´ ĠìĤ¬ëĬĶ ĠëıĻë¬¼ ìĿ´ëĿ¼ëĬĶ Ġê°ľë...,50대,DCDG20000009.json,10,test
...,...,...,...,...,...,...
227993,ê·¸ëŁ¬ë©´ìĦľ ĠìĿ¸ìłľ ĠìĿ´ëŁ° Ġìĸĺê¸°ëıĦ ĠìŀĪìŀ...,ê·¸ëŁ¬ë©´ìĦľ ĠìĿ¸ìłľ ĠìĿ´ëŁ° Ġìĸĺê¸°ëıĦ ĠìŀĪìŀ...,50대,DCNA20000036.json,5,test
227995,ê·¸ëŁ° Ġë§ĲìĿ´ ĠìĻĢëĭ¿ ìķĦìļĶ ĠìĿ¸ìłľ ĠëĤĺìĿ´ê...,ê·¸ëŁ° Ġë§ĲìĿ´ ĠìĻĢëĭ¿ ìķĦìļĶ ĠìĿ¸ìłľ ĠëĤĺìĿ´ê...,50대,DCNA20000036.json,7,test
227999,ìĿ¸ìłľ Ġì²ł ëĵ¤ ìŀĲ Ġìŀī ~ ĠìłĢê¸° Ġíķľëĭ¤ê³ł ...,ìĿ¸ìłľ Ġì²ł ëĵ¤ ìŀĲ Ġìŀī ~ ĠìłĢê¸° Ġíķľëĭ¤ê³ł ...,50대,DCNA20000036.json,9,test
228001,ìĸ´ëĸ»ê²Į Ġë³´ë©´ ĠìĦłìĥĿëĭĺ Ġì§Ģê¸Ī Ġê·¸ê²Į Ġ...,ìĸ´ëĸ»ê²Į Ġë³´ë©´ ĠìĦłìĥĿëĭĺ Ġì§Ģê¸Ī Ġê·¸ê²Į Ġ...,50대,DCNA20000036.json,11,test


In [94]:
bpe_tokenizer.from_file("../model_storage/stan-CC_bpe/test_vocab.json", "../model_storage/stan-CC_bpe/test_merges.txt")

for i in range(len(df)):
    is_unk = False
    dia = df.iloc[i, 0]
    token_ids = [bpe_tokenizer.token_to_id(token) for token in dia.split()]
    df.iloc[i, 0] = bpe_tokenizer.decode([token for token in token_ids if token is not None])
    
    stan = df.iloc[i, 1]
    token_ids = [bpe_tokenizer.token_to_id(token) for token in stan.split()]
    df.iloc[i, 1] = bpe_tokenizer.decode([token for token in token_ids if token is not None])

print(df)

                                                       방언  \
0                                       우리집 지금 십팔 십구 되잖아.   
2       어~ 아파서 고비가 있었는데 지금은 그걸 잘 극복해 가지고 건강하게 우리 식구랑 같...   
4                                       그래도 가족 일원으로서 예쁘지?   
5        아~ 예쁘긴 한데 엄마는 너무 나보다는이를 좀 더 많이 챙기지 않나 하는 생각이 드는데   
9                  그냥 같이 사는 동물이라는 개념으로 별로 그렇게 정이 가진 않았거든?   
...                                                   ...   
227993                               그러면서 인제 이런 얘기도 있잖아요.   
227995                           그런 말이 와닿아요 인제 나이가 인제 이렇게   
227999                     인제 철들자 잉~ 저기 한다고 쪼꼼 철이 드는지 어쩐지   
228001           어떻게 보면 선생님 지금 그게  오히려 인제 마음이 여유로워 졌기 때문에   
228002                    이게 좀 넓게 볼 수 있는게 아닐까 그런 생각도 들어요.   

                                                      표준어  연령대  \
0                                       우리집 지금 십팔 십구 되잖아.  50대   
2       어~ 아파서 고비가 있었는데 지금은 그걸 잘 극복해 가지고 건강하게 우리 식구랑 같...  50대   
4                                       그래도 가족 일원으로서 예쁘지?  50대   
5  

In [95]:
bpe_tokenizer.from_file("./vocab.json", "./merges.txt")

Tokenizer(vocabulary_size=30000, model=ByteLevelBPE, add_prefix_space=False, lowercase=False, dropout=None, unicode_normalizer=None, continuing_subword_prefix=None, end_of_word_suffix=None, trim_offsets=False)

In [96]:
code_result_dialect = bpe_tokenizer.encode_batch(df["방언"])
code_result_standard = bpe_tokenizer.encode_batch(df["표준어"])

In [97]:
df.loc[df.index, "방언"] = [' '.join(res.tokens) for res in code_result_dialect]
df["방언"]

0           ìļ°ë¦¬ ì§ĳ Ġì§Ģê¸Ī ĠìĭŃíĮĶ ĠìĭŃêµ¬ ĠëĲĺìŀĸìķĦ .
2         ìĸ´ ~ ĠìķĦíĮĮìĦľ Ġê³ł ë¹Ħê°Ģ ĠìŀĪìĹĪëĬĶëį° Ġì§...
4         ê·¸ëŀĺëıĦ Ġê°Ģì¡± ĠìĿ¼ ìĽĲ ìľ¼ë¡ľìĦľ ĠìĺĪìģĺì§Ģ ?
5         ìķĦ ~ ĠìĺĪìģĺ ê¸´ Ġíķľëį° ĠìĹĦë§ĪëĬĶ ĠëĦĪë¬´ Ġ...
9         ê·¸ëĥ¥ Ġê°ĻìĿ´ ĠìĤ¬ëĬĶ ĠëıĻë¬¼ ìĿ´ëĿ¼ëĬĶ Ġê°ľë...
                                ...                        
227993    ê·¸ëŁ¬ë©´ìĦľ ĠìĿ¸ìłľ ĠìĿ´ëŁ° Ġìĸĺê¸°ëıĦ ĠìŀĪìŀ...
227995    ê·¸ëŁ° Ġë§ĲìĿ´ ĠìĻĢëĭ¿ ìķĦìļĶ ĠìĿ¸ìłľ ĠëĤĺìĿ´ê...
227999    ìĿ¸ìłľ Ġì²ł ëĵ¤ ìŀĲ Ġìŀī ~ ĠìłĢê¸° Ġíķľëĭ¤ê³ł ...
228001    ìĸ´ëĸ»ê²Į Ġë³´ë©´ ĠìĦłìĥĿëĭĺ Ġì§Ģê¸Ī Ġê·¸ê²Į Ġ...
228002    ìĿ´ê²Į Ġì¢Ģ ĠëĦĵê²Į Ġë³¼ ĠìĪĺ ĠìŀĪëĬĶê²Į ĠìķĦë...
Name: 방언, Length: 95001, dtype: object

In [98]:
df.loc[df.index, "표준어"] = [' '.join(res.tokens) for res in code_result_standard]
df["표준어"]

0           ìļ°ë¦¬ ì§ĳ Ġì§Ģê¸Ī ĠìĭŃíĮĶ ĠìĭŃêµ¬ ĠëĲĺìŀĸìķĦ .
2         ìĸ´ ~ ĠìķĦíĮĮìĦľ Ġê³ł ë¹Ħê°Ģ ĠìŀĪìĹĪëĬĶëį° Ġì§...
4         ê·¸ëŀĺëıĦ Ġê°Ģì¡± ĠìĿ¼ ìĽĲ ìľ¼ë¡ľìĦľ ĠìĺĪìģĺì§Ģ ?
5         ìķĦ ~ ĠìĺĪìģĺ ê¸´ Ġíķľëį° ĠìĹĦë§ĪëĬĶ ĠëĦĪë¬´ Ġ...
9         ê·¸ëĥ¥ Ġê°ĻìĿ´ ĠìĤ¬ëĬĶ ĠëıĻë¬¼ ìĿ´ëĿ¼ëĬĶ Ġê°ľë...
                                ...                        
227993    ê·¸ëŁ¬ë©´ìĦľ ĠìĿ¸ìłľ ĠìĿ´ëŁ° Ġìĸĺê¸°ëıĦ ĠìŀĪìŀ...
227995    ê·¸ëŁ° Ġë§ĲìĿ´ ĠìĻĢëĭ¿ ìķĦìļĶ ĠìĿ¸ìłľ ĠëĤĺìĿ´ê...
227999    ìĿ¸ìłľ Ġì²ł ëĵ¤ ìŀĲ Ġìŀī ~ ĠìłĢê¸° Ġíķľëĭ¤ê³ł ...
228001    ìĸ´ëĸ»ê²Į Ġë³´ë©´ ĠìĦłìĥĿëĭĺ Ġì§Ģê¸Ī Ġê·¸ê²Į Ġ...
228002    ìĿ´ê²Į Ġì¢Ģ ĠëĦĵê²Į Ġë³¼ ĠìĪĺ ĠìŀĪëĬĶê²Į ĠìķĦë...
Name: 표준어, Length: 95001, dtype: object

In [99]:
ori_df = pd.read_csv("../datas/output/chungcheong_dialect_bpe_integration.csv", index_col=0)

ori_df[ori_df["셋"] == "test"] = df
ori_df[ori_df["셋"] == "test"]

,방언,표준어,연령대,출처 파일,길이,셋
0,ìļ°ë¦¬ ì§ĳ Ġì§Ģê¸Ī ĠìĭŃíĮĶ ĠìĭŃêµ¬ ĠëĲĺìŀĸìķĦ .,ìļ°ë¦¬ ì§ĳ Ġì§Ģê¸Ī ĠìĭŃíĮĶ ĠìĭŃêµ¬ ĠëĲĺìŀĸìķĦ .,50대,DCDG20000009.json,9,test
2,ìĸ´ ~ ĠìķĦíĮĮìĦľ Ġê³ł ë¹Ħê°Ģ ĠìŀĪìĹĪëĬĶëį° Ġì§...,ìĸ´ ~ ĠìķĦíĮĮìĦľ Ġê³ł ë¹Ħê°Ģ ĠìŀĪìĹĪëĬĶëį° Ġì§...,50대,DCDG20000009.json,17,test
4,ê·¸ëŀĺëıĦ Ġê°Ģì¡± ĠìĿ¼ ìĽĲ ìľ¼ë¡ľìĦľ ĠìĺĪìģĺì§Ģ ?,ê·¸ëŀĺëıĦ Ġê°Ģì¡± ĠìĿ¼ ìĽĲ ìľ¼ë¡ľìĦľ ĠìĺĪìģĺì§Ģ ?,50대,DCDG20000009.json,5,test
5,ìķĦ ~ ĠìĺĪìģĺ ê¸´ Ġíķľëį° ĠìĹĦë§ĪëĬĶ ĠëĦĪë¬´ Ġ...,ìķĦ ~ ĠìĺĪìģĺ ê¸´ Ġíķľëį° ĠìĹĦë§ĪëĬĶ ĠëĦĪë¬´ Ġ...,20대,DCDG20000009.json,15,test
9,ê·¸ëĥ¥ Ġê°ĻìĿ´ ĠìĤ¬ëĬĶ ĠëıĻë¬¼ ìĿ´ëĿ¼ëĬĶ Ġê°ľë...,ê·¸ëĥ¥ Ġê°ĻìĿ´ ĠìĤ¬ëĬĶ ĠëıĻë¬¼ ìĿ´ëĿ¼ëĬĶ Ġê°ľë...,50대,DCDG20000009.json,10,test
...,...,...,...,...,...,...
227993,ê·¸ëŁ¬ë©´ìĦľ ĠìĿ¸ìłľ ĠìĿ´ëŁ° Ġìĸĺê¸°ëıĦ ĠìŀĪìŀ...,ê·¸ëŁ¬ë©´ìĦľ ĠìĿ¸ìłľ ĠìĿ´ëŁ° Ġìĸĺê¸°ëıĦ ĠìŀĪìŀ...,50대,DCNA20000036.json,5,test
227995,ê·¸ëŁ° Ġë§ĲìĿ´ ĠìĻĢëĭ¿ ìķĦìļĶ ĠìĿ¸ìłľ ĠëĤĺìĿ´ê...,ê·¸ëŁ° Ġë§ĲìĿ´ ĠìĻĢëĭ¿ ìķĦìļĶ ĠìĿ¸ìłľ ĠëĤĺìĿ´ê...,50대,DCNA20000036.json,7,test
227999,ìĿ¸ìłľ Ġì²ł ëĵ¤ ìŀĲ Ġìŀī ~ ĠìłĢê¸° Ġíķľëĭ¤ê³ł ...,ìĿ¸ìłľ Ġì²ł ëĵ¤ ìŀĲ Ġìŀī ~ ĠìłĢê¸° Ġíķľëĭ¤ê³ł ...,50대,DCNA20000036.json,9,test
228001,ìĸ´ëĸ»ê²Į Ġë³´ë©´ ĠìĦłìĥĿëĭĺ Ġì§Ģê¸Ī Ġê·¸ê²Į Ġ...,ìĸ´ëĸ»ê²Į Ġë³´ë©´ ĠìĦłìĥĿëĭĺ Ġì§Ģê¸Ī Ġê·¸ê²Į Ġ...,50대,DCNA20000036.json,11,test


In [100]:
ori_df.to_csv("../datas/output/chungcheong_dialect_bpe_integration.csv")

### SentencePiece

In [110]:
df = pd.read_csv("../datas/output/chungcheong_dialect_SentencePiece_integration.csv", index_col=0)
df

,방언,표준어,연령대,출처 파일,길이,셋
0,▁영화 보다는 ▁그런 ▁쪽으로 ▁쫌 ▁많이 ▁다녀서,▁영화 보다는 ▁그런 ▁쪽으로 ▁조금 ▁많이 ▁다녀서,50대,DCNA20001462.json,6,train
1,▁에 ~ ▁저는 ▁원래 ▁고등학생 ▁때까지,▁에 ~ ▁저는 ▁원래 ▁고등학생 ▁때까지,20대,DCDG20000795.json,5,train
2,▁그 ▁companyname ▁프로그램 ▁이걸 ▁많이 ▁보거든 .,▁그 ▁companyname ▁프로그램 ▁이걸 ▁많이 ▁보거든 .,20대,DCDG20001601.json,6,train
3,▁고 위 험 ▁그 ~ ▁군 이 ▁아니야 ▁근데 ▁만약에 ▁그게 ▁임신 성 ▁당뇨 로...,▁고 위 험 ▁그 ~ ▁군 이 ▁아니야 ▁근데 ▁만약에 ▁그게 ▁임신 성 ▁당뇨 로...,40대,DCEX20001058.json,21,train
4,▁코로나 ▁때문에 ▁사람들이 ▁많이 ▁집에 ▁있고 ▁이제 ▁이릏게 ▁했으니까,▁코로나 ▁때문에 ▁사람들이 ▁많이 ▁집에 ▁있고 ▁이제 ▁이렇게 ▁했으니까,40대,DCNA20001390.json,9,train
...,...,...,...,...,...,...
227987,▁남 의 ▁인제 ▁그 ▁들 녘 을 ▁쳐다보 잖아요 ▁그러면 ▁정말 ▁벼 보다는,▁남 의 ▁인제 ▁그 ▁들 녘 을 ▁쳐다보 잖아요 ▁그러면 ▁정말 ▁벼 보다는,50대,DCNA20000036.json,8,test
227988,▁그 ▁잡 초 가 ▁더 ▁많이 ▁나와 ▁있는 ▁데가 ▁있잖아요 ▁보면,▁그 ▁잡 초 가 ▁더 ▁많이 ▁나와 ▁있는 ▁데가 ▁있잖아요 ▁보면,50대,DCNA20000036.json,9,test
227993,▁그러면서 ▁인제 ▁이런 ▁얘기 도 ▁있잖아요 .,▁그러면서 ▁인제 ▁이런 ▁얘기 도 ▁있잖아요 .,50대,DCNA20000036.json,5,test
227994,▁아유 ▁저기 ▁뭐야 ▁엉 ▁곡식 은 ▁주인 의 ▁발걸음 ▁소리 ▁듣고 ▁큰 다 .,▁아유 ▁저기 ▁뭐야 ▁엉 ▁곡식 은 ▁주인 의 ▁발걸음 ▁소리 ▁듣고 ▁큰 다 .,50대,DCNA20000036.json,10,test


In [111]:
df = df[df["셋"] == "test"]
df

,방언,표준어,연령대,출처 파일,길이,셋
0,▁우리집 ▁거북이 ▁용봉이가 ▁지금 ▁십팔 ▁년 에서 ▁십구 ▁년차 ▁되잖아 .,▁우리집 ▁거북이 ▁용봉이가 ▁지금 ▁십팔 ▁년 에서 ▁십구 ▁년차 ▁되잖아 .,50대,DCDG20000009.json,9,test
2,▁어 ~ ▁아파서 ▁고 비가 ▁있었는데 ▁지금은 ▁그걸 ▁잘 ▁극복해 ▁가지고 ▁건...,▁어 ~ ▁아파서 ▁고 비가 ▁있었는데 ▁지금은 ▁그걸 ▁잘 ▁극복해 ▁가지고 ▁건...,50대,DCDG20000009.json,17,test
3,▁근데 ▁엄마가 ▁너무 ▁용봉이 를 ▁예뻐 한다고 ▁니가 ▁엄청 ▁질투 가 ▁많은데,▁근데 ▁엄마가 ▁너무 ▁용봉이 를 ▁예뻐 한다고 ▁니가 ▁엄청 ▁질투 가 ▁많은데,50대,DCDG20000009.json,9,test
4,▁그래도 ▁용봉이가 ▁가족 ▁일 원 으로서 ▁예쁘지 ?,▁그래도 ▁용봉이가 ▁가족 ▁일 원 으로서 ▁예쁘지 ?,50대,DCDG20000009.json,5,test
8,▁용봉이가 ▁엄마가 ▁처음에는 ▁한 ▁십 여 ▁년 ▁동안은,▁용봉이가 ▁엄마가 ▁처음에는 ▁한 ▁십 여 ▁년 ▁동안은,50대,DCDG20000009.json,7,test
...,...,...,...,...,...,...
227987,▁남 의 ▁인제 ▁그 ▁들 녘 을 ▁쳐다보 잖아요 ▁그러면 ▁정말 ▁벼 보다는,▁남 의 ▁인제 ▁그 ▁들 녘 을 ▁쳐다보 잖아요 ▁그러면 ▁정말 ▁벼 보다는,50대,DCNA20000036.json,8,test
227988,▁그 ▁잡 초 가 ▁더 ▁많이 ▁나와 ▁있는 ▁데가 ▁있잖아요 ▁보면,▁그 ▁잡 초 가 ▁더 ▁많이 ▁나와 ▁있는 ▁데가 ▁있잖아요 ▁보면,50대,DCNA20000036.json,9,test
227993,▁그러면서 ▁인제 ▁이런 ▁얘기 도 ▁있잖아요 .,▁그러면서 ▁인제 ▁이런 ▁얘기 도 ▁있잖아요 .,50대,DCNA20000036.json,5,test
227994,▁아유 ▁저기 ▁뭐야 ▁엉 ▁곡식 은 ▁주인 의 ▁발걸음 ▁소리 ▁듣고 ▁큰 다 .,▁아유 ▁저기 ▁뭐야 ▁엉 ▁곡식 은 ▁주인 의 ▁발걸음 ▁소리 ▁듣고 ▁큰 다 .,50대,DCNA20000036.json,10,test


In [113]:
sp_tokenizer = spm.SentencePieceProcessor(model_file='../model_storage/stan-CC_SP/chungcheong_sp.model')

In [114]:
df.loc[df.index, "방언"] = [sp_tokenizer.decode(sentence.split()) for sentence in df["방언"]]
df.loc[df.index, "표준어"] = [sp_tokenizer.decode(sentence.split()) for sentence in df["표준어"]]

df

,방언,표준어,연령대,출처 파일,길이,셋
0,우리집 거북이 용봉이가 지금 십팔 년에서 십구 년차 되잖아.,우리집 거북이 용봉이가 지금 십팔 년에서 십구 년차 되잖아.,50대,DCDG20000009.json,9,test
2,어~ 아파서 고비가 있었는데 지금은 그걸 잘 극복해 가지고 건강하게 우리 식구랑 같...,어~ 아파서 고비가 있었는데 지금은 그걸 잘 극복해 가지고 건강하게 우리 식구랑 같...,50대,DCDG20000009.json,17,test
3,근데 엄마가 너무 용봉이를 예뻐한다고 니가 엄청 질투가 많은데,근데 엄마가 너무 용봉이를 예뻐한다고 니가 엄청 질투가 많은데,50대,DCDG20000009.json,9,test
4,그래도 용봉이가 가족 일원으로서 예쁘지?,그래도 용봉이가 가족 일원으로서 예쁘지?,50대,DCDG20000009.json,5,test
8,용봉이가 엄마가 처음에는 한 십여 년 동안은,용봉이가 엄마가 처음에는 한 십여 년 동안은,50대,DCDG20000009.json,7,test
...,...,...,...,...,...,...
227987,남의 인제 그 들녘을 쳐다보잖아요 그러면 정말 벼보다는,남의 인제 그 들녘을 쳐다보잖아요 그러면 정말 벼보다는,50대,DCNA20000036.json,8,test
227988,그 잡초가 더 많이 나와 있는 데가 있잖아요 보면,그 잡초가 더 많이 나와 있는 데가 있잖아요 보면,50대,DCNA20000036.json,9,test
227993,그러면서 인제 이런 얘기도 있잖아요.,그러면서 인제 이런 얘기도 있잖아요.,50대,DCNA20000036.json,5,test
227994,아유 저기 뭐야 엉 곡식은 주인의 발걸음 소리 듣고 큰다.,아유 저기 뭐야 엉 곡식은 주인의 발걸음 소리 듣고 큰다.,50대,DCNA20000036.json,10,test


In [116]:
sp_tokenizer = spm.SentencePieceProcessor(model_file='../model_storage/stan-CC_SP/chungcheong_data_sp.model')

code_result_dialect = sp_tokenizer.encode(df["방언"].to_list(), out_type=str)
code_result_standard = sp_tokenizer.encode(df["표준어"].to_list(), out_type=str)

In [117]:
df.loc[df.index, "방언"] = [' '.join(res) for res in code_result_dialect]
df["방언"]

0            ▁우리집 ▁거북이 ▁용 봉 이가 ▁지금 ▁십팔 ▁년 에서 ▁십구 ▁년차 ▁되잖아 .
2         ▁어 ~ ▁아파서 ▁고비 가 ▁있었는데 ▁지금은 ▁그걸 ▁잘 ▁극복해 ▁가지고 ▁건...
3          ▁근데 ▁엄마가 ▁너무 ▁용 봉 이 를 ▁예뻐 한다고 ▁니가 ▁엄청 ▁질투 가 ▁많은데
4                          ▁그래도 ▁용 봉 이가 ▁가족 ▁일 원으로 서 ▁예쁘지 ?
8                        ▁용 봉 이가 ▁엄마가 ▁처음에는 ▁한 ▁십 여 ▁년 ▁동안은
                                ...                        
227987           ▁남의 ▁인제 ▁그 ▁들 녘 을 ▁쳐다보 잖아요 ▁그러면 ▁정말 ▁벼 보다는
227988              ▁그 ▁ 잡 초 가 ▁더 ▁많이 ▁나와 ▁있는 ▁데가 ▁있잖아요 ▁보면
227993                           ▁그러면서 ▁인제 ▁이런 ▁얘기도 ▁있잖아요 .
227994       ▁아유 ▁저기 ▁뭐야 ▁엉 ▁곡식 은 ▁주인 의 ▁발걸음 ▁소리 ▁듣고 ▁큰 다 .
227995                   ▁그런 ▁말이 ▁와 닿 아 요 ▁인제 ▁나이가 ▁인제 ▁이렇게
Name: 방언, Length: 95000, dtype: object

In [118]:
df.loc[df.index, "표준어"] = [' '.join(res) for res in code_result_standard]
df["표준어"]

0            ▁우리집 ▁거북이 ▁용 봉 이가 ▁지금 ▁십팔 ▁년 에서 ▁십구 ▁년차 ▁되잖아 .
2         ▁어 ~ ▁아파서 ▁고비 가 ▁있었는데 ▁지금은 ▁그걸 ▁잘 ▁극복해 ▁가지고 ▁건...
3          ▁근데 ▁엄마가 ▁너무 ▁용 봉 이 를 ▁예뻐 한다고 ▁니가 ▁엄청 ▁질투 가 ▁많은데
4                          ▁그래도 ▁용 봉 이가 ▁가족 ▁일 원으로 서 ▁예쁘지 ?
8                        ▁용 봉 이가 ▁엄마가 ▁처음에는 ▁한 ▁십 여 ▁년 ▁동안은
                                ...                        
227987           ▁남의 ▁인제 ▁그 ▁들 녘 을 ▁쳐다보 잖아요 ▁그러면 ▁정말 ▁벼 보다는
227988              ▁그 ▁ 잡 초 가 ▁더 ▁많이 ▁나와 ▁있는 ▁데가 ▁있잖아요 ▁보면
227993                           ▁그러면서 ▁인제 ▁이런 ▁얘기도 ▁있잖아요 .
227994       ▁아유 ▁저기 ▁뭐야 ▁엉 ▁곡식 은 ▁주인 의 ▁발걸음 ▁소리 ▁듣고 ▁큰 다 .
227995                   ▁그런 ▁말이 ▁와 닿 아 요 ▁인제 ▁나이가 ▁인제 ▁이렇게
Name: 표준어, Length: 95000, dtype: object

In [119]:
ori_df = pd.read_csv("../datas/output/chungcheong_dialect_SentencePiece_integration.csv", index_col=0)

ori_df[ori_df["셋"] == "test"] = df
ori_df[ori_df["셋"] == "test"]

,방언,표준어,연령대,출처 파일,길이,셋
0,▁우리집 ▁거북이 ▁용 봉 이가 ▁지금 ▁십팔 ▁년 에서 ▁십구 ▁년차 ▁되잖아 .,▁우리집 ▁거북이 ▁용 봉 이가 ▁지금 ▁십팔 ▁년 에서 ▁십구 ▁년차 ▁되잖아 .,50대,DCDG20000009.json,9,test
2,▁어 ~ ▁아파서 ▁고비 가 ▁있었는데 ▁지금은 ▁그걸 ▁잘 ▁극복해 ▁가지고 ▁건...,▁어 ~ ▁아파서 ▁고비 가 ▁있었는데 ▁지금은 ▁그걸 ▁잘 ▁극복해 ▁가지고 ▁건...,50대,DCDG20000009.json,17,test
3,▁근데 ▁엄마가 ▁너무 ▁용 봉 이 를 ▁예뻐 한다고 ▁니가 ▁엄청 ▁질투 가 ▁많은데,▁근데 ▁엄마가 ▁너무 ▁용 봉 이 를 ▁예뻐 한다고 ▁니가 ▁엄청 ▁질투 가 ▁많은데,50대,DCDG20000009.json,9,test
4,▁그래도 ▁용 봉 이가 ▁가족 ▁일 원으로 서 ▁예쁘지 ?,▁그래도 ▁용 봉 이가 ▁가족 ▁일 원으로 서 ▁예쁘지 ?,50대,DCDG20000009.json,5,test
8,▁용 봉 이가 ▁엄마가 ▁처음에는 ▁한 ▁십 여 ▁년 ▁동안은,▁용 봉 이가 ▁엄마가 ▁처음에는 ▁한 ▁십 여 ▁년 ▁동안은,50대,DCDG20000009.json,7,test
...,...,...,...,...,...,...
227987,▁남의 ▁인제 ▁그 ▁들 녘 을 ▁쳐다보 잖아요 ▁그러면 ▁정말 ▁벼 보다는,▁남의 ▁인제 ▁그 ▁들 녘 을 ▁쳐다보 잖아요 ▁그러면 ▁정말 ▁벼 보다는,50대,DCNA20000036.json,8,test
227988,▁그 ▁ 잡 초 가 ▁더 ▁많이 ▁나와 ▁있는 ▁데가 ▁있잖아요 ▁보면,▁그 ▁ 잡 초 가 ▁더 ▁많이 ▁나와 ▁있는 ▁데가 ▁있잖아요 ▁보면,50대,DCNA20000036.json,9,test
227993,▁그러면서 ▁인제 ▁이런 ▁얘기도 ▁있잖아요 .,▁그러면서 ▁인제 ▁이런 ▁얘기도 ▁있잖아요 .,50대,DCNA20000036.json,5,test
227994,▁아유 ▁저기 ▁뭐야 ▁엉 ▁곡식 은 ▁주인 의 ▁발걸음 ▁소리 ▁듣고 ▁큰 다 .,▁아유 ▁저기 ▁뭐야 ▁엉 ▁곡식 은 ▁주인 의 ▁발걸음 ▁소리 ▁듣고 ▁큰 다 .,50대,DCNA20000036.json,10,test


In [120]:
ori_df.to_csv("../datas/output/chungcheong_dialect_SentencePiece_integration.csv")

## 경상 방언

### BPE

In [121]:
df = pd.read_csv("../datas/output/gyeongsang_dialect_bpe_integration.csv", index_col=0)
df

,방언,표준어,연령대,출처 파일,길이,셋
0,ìķĦ ~ ĠìĿ´ Ġëħ¸ëŀĺëĬĶ Ġëħ¸ëŀĺëĬĶ Ġì¢ĭìŀĸìķĦ Ġì...,ìķĦ ~ ĠìĿ´ Ġëħ¸ëŀĺëĬĶ Ġëħ¸ëŀĺëĬĶ Ġì¢ĭìŀĸìķĦ Ġì...,20대,DKSR20002079.json,7,train
1,ëķħ ìĿ´ëĤĺ ĠìĿ´ëŁ° ĠëĤĺë¬´ë ĵ¤ìĿ´ ĠìĿ¸ìŀĲ Ġë§ī...,ëķħ ìĿ´ëĤĺ ĠìĿ´ëŁ° ĠëĤĺë¬´ë ĵ¤ìĿ´ ĠìĿ´ìłľ Ġë§ī...,50대,DKCI20002541.json,8,train
2,ìĸ´ì ¨Į ê±´ ĠìķĦì¹¨ìĹĲ Ġë§¤ìĿ¼ ĠìĿ¼ì°į ĠìĿ¼ìĸ´...,ìĸ´ì ¨Į ê±´ ĠìķĦì¹¨ìĹĲ Ġë§¤ìĿ¼ ĠìĿ¼ì°į ĠìĿ¼ìĸ´...,50대,DKCI20003110.json,18,train
3,ìĥĿíĻľ Ġë³´ë©´ Ġëĭ¤ Ġëĵľ ë¦ ħ ëĭ¤ Ġì¹´ëĮĢ Ġê·¸...,ìĥĿíĻľ Ġë³´ë©´ Ġëĭ¤ ĠëįĶëŁ½ ëĭ¤ ĠíķĺëĮĢ Ġê·¸ëŀ...,20대,DKSR20000643.json,6,train
4,ê·¸ëĥ¥ Ġë°Ķë¥´ì ŀĲ ëĭĪ ĠëĺĲ Ġê°ģ ì§ĪìĿĢ ĠëĲĺê³ł,ê·¸ëĥ¥ Ġë°Ķë¥´ì ŀĲ ëĭĪ ĠëĺĲ Ġê°ģ ì§ĪìĿĢ ĠëĲĺê³ł,10대,DKSR20003850.json,5,train
...,...,...,...,...,...,...
229489,ìłľìĿ¼ Ġë³Ħë¡ľìĺĢëįĺ Ġìĩ¼íķĳ ĠìłľìĿ¼ Ġì¢ĭìķĺëį...,ìłľìĿ¼ Ġë³Ħë¡ľìĺĢëįĺ Ġìĩ¼íķĳ ĠìłľìĿ¼ Ġì¢ĭìķĺëį...,20대,DKSR20001736.json,9,test
229490,ìłľìĿ¼ Ġë³Ħë¡ľìĺĢëįĺ Ġê±°ëĬĶ Ġê·¸ ~ ĠëıĻëĤ¨ìķĦ...,ìłľìĿ¼ Ġë³Ħë¡ľìĺĢëįĺ Ġê±°ëĬĶ Ġê·¸ ~ ĠëıĻëĤ¨ìķĦ...,20대,DKSR20001736.json,7,test
229493,ëĴ¤ìĹĲ ĠëĤ´ê°Ģ ĠíĥĢëĬĶ Ġê±° Ġê·¸ëŁ° Ġê±° ĠëŃĲë...,ëĴ¤ìĹĲ ĠëĤ´ê°Ģ ĠíĥĢëĬĶ Ġê±° Ġê·¸ëŁ° Ġê±° ĠëŃĲë...,20대,DKSR20001736.json,9,test
229494,ìĸ´ì¨Įëĵł Ġê·¸ëŁ° Ġìķ½ê°Ħ ĠìĿ´ëıĻ ĠìĪĺ ëĭ¨ìĿĦ ...,ìĸ´ì¨Įëĵł Ġê·¸ëŁ° Ġìķ½ê°Ħ ĠìĿ´ëıĻ ĠìĪĺ ëĭ¨ìĿĦ ...,20대,DKSR20001736.json,6,test


In [123]:
from tokenizers import ByteLevelBPETokenizer

bpe_tokenizer = ByteLevelBPETokenizer()
bpe_tokenizer.train(
    files="../datas/output/gyeongsang_dialect_data_text.txt",
)
bpe_tokenizer.save_model("./")

['./vocab.json', './merges.txt']

In [124]:
df = df[df["셋"] == "test"]
df

,방언,표준어,연령대,출처 파일,길이,셋
4,ìµľ ìĨĮíķľ ìĿĺ Ġë§Īì§Ģëħ¸ ìĦłìĿ´ ĠëĤĺë³´ëĭ¤ ë§...,ìµľ ìĨĮíķľ ìĿĺ Ġë§Īì§Ģëħ¸ ìĦłìĿ´ ĠëĤĺë³´ëĭ¤ ë§...,20대,DKSR20000890.json,8,test
5,íģ° Ġê²Į Ġì¢ĭìŀĸìķĦìļĶ Ġìķ½ê°Ħ ĠëĤ¨ëĵ¤ìĿ´ Ġë³¼...,íģ° Ġê²Į Ġì¢ĭìŀĸìķĦìļĶ Ġìķ½ê°Ħ ĠëĤ¨ëĵ¤ìĿ´ Ġë³¼...,20대,DKSR20000890.json,7,test
6,ìĿ´ìĥģ ìłģìĿ¸ ĠíĤ¤ ëĿ¼ëĬĶ Ġê²Į ĠìŀĪìľ¼ëĭĪê¹Į Ġ...,ìĿ´ìĥģ ìłģìĿ¸ ĠíĤ¤ ëĿ¼ëĬĶ Ġê²Į ĠìŀĪìľ¼ëĭĪê¹Į Ġ...,20대,DKSR20000890.json,5,test
8,ëĶ± Ġì¢ĭìĿĢ Ġê±° Ġê°Ļê³ł ĠìĿ¼ëĭ¨ Ġê·¸ê²Į Ġê²īì...,ëĶ± Ġì¢ĭìĿĢ Ġê±° Ġê°Ļê³ł ĠìĿ¼ëĭ¨ Ġê·¸ê²Į Ġê²īì...,20대,DKSR20000890.json,9,test
12,ìĦ±ê²©ìĿ´ ĠìĿ¼ëĭ¨ Ġìłľê°Ģ Ġì«Į Ġë§ī Ġëĵľ ìĦ¼ Ġ...,ìĦ±ê²©ìĿ´ ĠìĿ¼ëĭ¨ Ġìłľê°Ģ Ġì¡°ê¸Ī Ġë§ī Ġëĵľ ìĦ...,20대,DKSR20000890.json,8,test
...,...,...,...,...,...,...
229489,ìłľìĿ¼ Ġë³Ħë¡ľìĺĢëįĺ Ġìĩ¼íķĳ ĠìłľìĿ¼ Ġì¢ĭìķĺëį...,ìłľìĿ¼ Ġë³Ħë¡ľìĺĢëįĺ Ġìĩ¼íķĳ ĠìłľìĿ¼ Ġì¢ĭìķĺëį...,20대,DKSR20001736.json,9,test
229490,ìłľìĿ¼ Ġë³Ħë¡ľìĺĢëįĺ Ġê±°ëĬĶ Ġê·¸ ~ ĠëıĻëĤ¨ìķĦ...,ìłľìĿ¼ Ġë³Ħë¡ľìĺĢëįĺ Ġê±°ëĬĶ Ġê·¸ ~ ĠëıĻëĤ¨ìķĦ...,20대,DKSR20001736.json,7,test
229493,ëĴ¤ìĹĲ ĠëĤ´ê°Ģ ĠíĥĢëĬĶ Ġê±° Ġê·¸ëŁ° Ġê±° ĠëŃĲë...,ëĴ¤ìĹĲ ĠëĤ´ê°Ģ ĠíĥĢëĬĶ Ġê±° Ġê·¸ëŁ° Ġê±° ĠëŃĲë...,20대,DKSR20001736.json,9,test
229494,ìĸ´ì¨Įëĵł Ġê·¸ëŁ° Ġìķ½ê°Ħ ĠìĿ´ëıĻ ĠìĪĺ ëĭ¨ìĿĦ ...,ìĸ´ì¨Įëĵł Ġê·¸ëŁ° Ġìķ½ê°Ħ ĠìĿ´ëıĻ ĠìĪĺ ëĭ¨ìĿĦ ...,20대,DKSR20001736.json,6,test


In [125]:
bpe_tokenizer.from_file("../model_storage/stan-GS_bpe/test_vocab.json", "../model_storage/stan-GS_bpe/test_merges.txt")

for i in range(len(df)):
    is_unk = False
    dia = df.iloc[i, 0]
    token_ids = [bpe_tokenizer.token_to_id(token) for token in dia.split()]
    df.iloc[i, 0] = bpe_tokenizer.decode([token for token in token_ids if token is not None])
    
    stan = df.iloc[i, 1]
    token_ids = [bpe_tokenizer.token_to_id(token) for token in stan.split()]
    df.iloc[i, 1] = bpe_tokenizer.decode([token for token in token_ids if token is not None])

print(df)

                                      방언                               표준어  \
4        최소한의선이 나보다만 크면 된다 그치만 그래도 이왕이면은   최소한의선이 나보다만 크면 된다 그치만 그래도 이왕이면은   
5                   큰 게 좋잖아요 약간 남들이 볼 때도              큰 게 좋잖아요 약간 남들이 볼 때도   
6                    이상적인 키라는 게 있으니까 그래서               이상적인 키라는 게 있으니까 그래서   
8            딱 좋은 거 같고 일단 그게 겉으로 보는 이상형?       딱 좋은 거 같고 일단 그게 겉으로 보는 이상형?   
12                성격이 일단 제가 쫌 막 드센 건 아닌데           성격이 일단 제가 조금 막 드센 건 아닌데   
...                                  ...                               ...   
229489  제일 별로였던 쇼핑 제일 좋았던 쇼핑이 어떤 게 있으세요?  제일 별로였던 쇼핑 제일 좋았던 쇼핑이 어떤 게 있으세요?   
229490            제일 별로였던 거는 그~ 동남아 갔을 때            제일 별로였던 거는 그~ 동남아 갔을 때   
229493      뒤에 내가 타는 거 그런 거 뭐라 하는지 모르겠는데      뒤에 내가 타는 거 그런 거 뭐라 하는지 모르겠는데   
229494              어쨌든 그런 약간 이동 수단을 탔는데              어쨌든 그런 약간 이동 수단을 탔는데   
229496       그거 타가지고 써가지고 그 싸웠던 게 생각납니다.       그거 타가지고 써가지고 그 싸웠던 게 생각납니다.   

        연령대              출처 파일  길이     셋  
4       20대  DKSR200

In [126]:
bpe_tokenizer.from_file("./vocab.json", "./merges.txt")

Tokenizer(vocabulary_size=30000, model=ByteLevelBPE, add_prefix_space=False, lowercase=False, dropout=None, unicode_normalizer=None, continuing_subword_prefix=None, end_of_word_suffix=None, trim_offsets=False)

In [127]:
code_result_dialect = bpe_tokenizer.encode_batch(df["방언"])
code_result_standard = bpe_tokenizer.encode_batch(df["표준어"])

In [128]:
df.loc[df.index, "방언"] = [' '.join(res.tokens) for res in code_result_dialect]
df["방언"]

4         ìµľ ìĨĮíķľ ìĿĺ ìĦłìĿ´ ĠëĤĺë³´ëĭ¤ ë§Į Ġíģ¬ë©´ Ġ...
5         íģ° Ġê²Į Ġì¢ĭìŀĸìķĦìļĶ Ġìķ½ê°Ħ ĠëĤ¨ëĵ¤ìĿ´ Ġë³¼...
6         ìĿ´ìĥģ ìłģìĿ¸ ĠíĤ¤ ëĿ¼ëĬĶ Ġê²Į ĠìŀĪìľ¼ëĭĪê¹Į Ġ...
8         ëĶ± Ġì¢ĭìĿĢ Ġê±° Ġê°Ļê³ł ĠìĿ¼ëĭ¨ Ġê·¸ê²Į Ġê²īì...
12        ìĦ±ê²©ìĿ´ ĠìĿ¼ëĭ¨ Ġìłľê°Ģ Ġì«Į Ġë§ī Ġëĵľ ìĦ¼ Ġ...
                                ...                        
229489    ìłľìĿ¼ Ġë³Ħë¡ľìĺĢëįĺ Ġìĩ¼íķĳ ĠìłľìĿ¼ Ġì¢ĭìķĺëį...
229490    ìłľìĿ¼ Ġë³Ħë¡ľìĺĢëįĺ Ġê±°ëĬĶ Ġê·¸ ~ ĠëıĻëĤ¨ìķĦ...
229493    ëĴ¤ìĹĲ ĠëĤ´ê°Ģ ĠíĥĢëĬĶ Ġê±° Ġê·¸ëŁ° Ġê±° ĠëŃĲë...
229494    ìĸ´ì¨Įëĵł Ġê·¸ëŁ° Ġìķ½ê°Ħ ĠìĿ´ëıĻ ĠìĪĺ ëĭ¨ìĿĦ ...
229496    ê·¸ê±° ĠíĥĢê°Ģì§Ģê³ł Ġìį¨ê°Ģì§Ģê³ł Ġê·¸ Ġìĭ¸ìĽ...
Name: 방언, Length: 89511, dtype: object

In [129]:
df.loc[df.index, "표준어"] = [' '.join(res.tokens) for res in code_result_standard]
df["표준어"]

4         ìµľ ìĨĮíķľ ìĿĺ ìĦłìĿ´ ĠëĤĺë³´ëĭ¤ ë§Į Ġíģ¬ë©´ Ġ...
5         íģ° Ġê²Į Ġì¢ĭìŀĸìķĦìļĶ Ġìķ½ê°Ħ ĠëĤ¨ëĵ¤ìĿ´ Ġë³¼...
6         ìĿ´ìĥģ ìłģìĿ¸ ĠíĤ¤ ëĿ¼ëĬĶ Ġê²Į ĠìŀĪìľ¼ëĭĪê¹Į Ġ...
8         ëĶ± Ġì¢ĭìĿĢ Ġê±° Ġê°Ļê³ł ĠìĿ¼ëĭ¨ Ġê·¸ê²Į Ġê²īì...
12        ìĦ±ê²©ìĿ´ ĠìĿ¼ëĭ¨ Ġìłľê°Ģ Ġì¡°ê¸Ī Ġë§ī Ġëĵľ ìĦ...
                                ...                        
229489    ìłľìĿ¼ Ġë³Ħë¡ľìĺĢëįĺ Ġìĩ¼íķĳ ĠìłľìĿ¼ Ġì¢ĭìķĺëį...
229490    ìłľìĿ¼ Ġë³Ħë¡ľìĺĢëįĺ Ġê±°ëĬĶ Ġê·¸ ~ ĠëıĻëĤ¨ìķĦ...
229493    ëĴ¤ìĹĲ ĠëĤ´ê°Ģ ĠíĥĢëĬĶ Ġê±° Ġê·¸ëŁ° Ġê±° ĠëŃĲë...
229494    ìĸ´ì¨Įëĵł Ġê·¸ëŁ° Ġìķ½ê°Ħ ĠìĿ´ëıĻ ĠìĪĺ ëĭ¨ìĿĦ ...
229496    ê·¸ê±° ĠíĥĢê°Ģì§Ģê³ł Ġìį¨ê°Ģì§Ģê³ł Ġê·¸ Ġìĭ¸ìĽ...
Name: 표준어, Length: 89511, dtype: object

In [130]:
ori_df = pd.read_csv("../datas/output/gyeongsang_dialect_bpe_integration.csv", index_col=0)

ori_df[ori_df["셋"] == "test"] = df
ori_df[ori_df["셋"] == "test"]

,방언,표준어,연령대,출처 파일,길이,셋
4,ìµľ ìĨĮíķľ ìĿĺ ìĦłìĿ´ ĠëĤĺë³´ëĭ¤ ë§Į Ġíģ¬ë©´ Ġ...,ìµľ ìĨĮíķľ ìĿĺ ìĦłìĿ´ ĠëĤĺë³´ëĭ¤ ë§Į Ġíģ¬ë©´ Ġ...,20대,DKSR20000890.json,8,test
5,íģ° Ġê²Į Ġì¢ĭìŀĸìķĦìļĶ Ġìķ½ê°Ħ ĠëĤ¨ëĵ¤ìĿ´ Ġë³¼...,íģ° Ġê²Į Ġì¢ĭìŀĸìķĦìļĶ Ġìķ½ê°Ħ ĠëĤ¨ëĵ¤ìĿ´ Ġë³¼...,20대,DKSR20000890.json,7,test
6,ìĿ´ìĥģ ìłģìĿ¸ ĠíĤ¤ ëĿ¼ëĬĶ Ġê²Į ĠìŀĪìľ¼ëĭĪê¹Į Ġ...,ìĿ´ìĥģ ìłģìĿ¸ ĠíĤ¤ ëĿ¼ëĬĶ Ġê²Į ĠìŀĪìľ¼ëĭĪê¹Į Ġ...,20대,DKSR20000890.json,5,test
8,ëĶ± Ġì¢ĭìĿĢ Ġê±° Ġê°Ļê³ł ĠìĿ¼ëĭ¨ Ġê·¸ê²Į Ġê²īì...,ëĶ± Ġì¢ĭìĿĢ Ġê±° Ġê°Ļê³ł ĠìĿ¼ëĭ¨ Ġê·¸ê²Į Ġê²īì...,20대,DKSR20000890.json,9,test
12,ìĦ±ê²©ìĿ´ ĠìĿ¼ëĭ¨ Ġìłľê°Ģ Ġì«Į Ġë§ī Ġëĵľ ìĦ¼ Ġ...,ìĦ±ê²©ìĿ´ ĠìĿ¼ëĭ¨ Ġìłľê°Ģ Ġì¡°ê¸Ī Ġë§ī Ġëĵľ ìĦ...,20대,DKSR20000890.json,8,test
...,...,...,...,...,...,...
229489,ìłľìĿ¼ Ġë³Ħë¡ľìĺĢëįĺ Ġìĩ¼íķĳ ĠìłľìĿ¼ Ġì¢ĭìķĺëį...,ìłľìĿ¼ Ġë³Ħë¡ľìĺĢëįĺ Ġìĩ¼íķĳ ĠìłľìĿ¼ Ġì¢ĭìķĺëį...,20대,DKSR20001736.json,9,test
229490,ìłľìĿ¼ Ġë³Ħë¡ľìĺĢëįĺ Ġê±°ëĬĶ Ġê·¸ ~ ĠëıĻëĤ¨ìķĦ...,ìłľìĿ¼ Ġë³Ħë¡ľìĺĢëįĺ Ġê±°ëĬĶ Ġê·¸ ~ ĠëıĻëĤ¨ìķĦ...,20대,DKSR20001736.json,7,test
229493,ëĴ¤ìĹĲ ĠëĤ´ê°Ģ ĠíĥĢëĬĶ Ġê±° Ġê·¸ëŁ° Ġê±° ĠëŃĲë...,ëĴ¤ìĹĲ ĠëĤ´ê°Ģ ĠíĥĢëĬĶ Ġê±° Ġê·¸ëŁ° Ġê±° ĠëŃĲë...,20대,DKSR20001736.json,9,test
229494,ìĸ´ì¨Įëĵł Ġê·¸ëŁ° Ġìķ½ê°Ħ ĠìĿ´ëıĻ ĠìĪĺ ëĭ¨ìĿĦ ...,ìĸ´ì¨Įëĵł Ġê·¸ëŁ° Ġìķ½ê°Ħ ĠìĿ´ëıĻ ĠìĪĺ ëĭ¨ìĿĦ ...,20대,DKSR20001736.json,6,test


In [131]:
ori_df.to_csv("../datas/output/gyeongsang_dialect_bpe_integration.csv")

### SentencePiece

In [133]:
df = pd.read_csv("../datas/output/gyeongsang_dialect_SentencePiece_integration.csv", index_col=0)
df

,방언,표준어,연령대,출처 파일,길이,셋
0,▁아 ~ ▁이 ▁노래는 ▁노래는 ▁좋잖아 ▁이런 ▁느낌이에요 .,▁아 ~ ▁이 ▁노래는 ▁노래는 ▁좋잖아 ▁이런 ▁느낌이에요 .,20대,DKSR20002079.json,7,train
1,▁땅 이나 ▁이런 ▁나무 들이 ▁인자 ▁막 ▁ 옴 을 ▁ 틔 워 ▁갖고,▁땅 이나 ▁이런 ▁나무 들이 ▁이제 ▁막 ▁ 옴 을 ▁ 틔 워 ▁갖고,50대,DKCI20002541.json,8,train
2,▁어 쨌 건 ▁아침에 ▁매일 ▁일찍 ▁일어나 가주고 ▁새벽 ▁다섯 ▁시 ▁되면 ▁일...,▁어 쨌 건 ▁아침에 ▁매일 ▁일찍 ▁일어나가지고 ▁새벽 ▁다섯 ▁시 ▁되면 ▁일어...,50대,DKCI20003110.json,18,train
3,▁생활 ▁보면 ▁다 ▁드 릅 다 ▁카대 ▁그래가지고,▁생활 ▁보면 ▁다 ▁더럽다 ▁하대 ▁그래가지고,20대,DKSR20000643.json,6,train
4,▁그냥 ▁바르 자니 ▁또 ▁각질 은 ▁되고,▁그냥 ▁바르 자니 ▁또 ▁각질 은 ▁되고,10대,DKSR20003850.json,5,train
...,...,...,...,...,...,...
229489,▁제일 ▁별로 였던 ▁쇼핑 ▁제일 ▁좋았던 ▁쇼핑이 ▁어떤 ▁게 ▁있으세요 ?,▁제일 ▁별로 였던 ▁쇼핑 ▁제일 ▁좋았던 ▁쇼핑이 ▁어떤 ▁게 ▁있으세요 ?,20대,DKSR20001736.json,9,test
229490,▁제일 ▁별로 였던 ▁거는 ▁그 ~ ▁동남아 ▁갔을 ▁때,▁제일 ▁별로 였던 ▁거는 ▁그 ~ ▁동남아 ▁갔을 ▁때,20대,DKSR20001736.json,7,test
229491,▁그 ~ ▁자전거 ▁같은 ▁거 ▁인력 거 ▁같은 ▁거 ?,▁그 ~ ▁자전거 ▁같은 ▁거 ▁인력 거 ▁같은 ▁거 ?,20대,DKSR20001736.json,7,test
229493,▁뒤에 ▁내가 ▁타는 ▁거 ▁그런 ▁거 ▁뭐라 ▁하는지 ▁모르겠는데,▁뒤에 ▁내가 ▁타는 ▁거 ▁그런 ▁거 ▁뭐라 ▁하는지 ▁모르겠는데,20대,DKSR20001736.json,9,test


In [134]:
df = df[df["셋"] == "test"]
df

,방언,표준어,연령대,출처 파일,길이,셋
3,▁정해놓 은 ▁건 ▁없는데 ▁일단 ▁제가 ▁키가 ▁되게 ▁큰 ▁편이여가지구,▁정해놓 은 ▁건 ▁없는데 ▁일단 ▁제가 ▁키가 ▁되게 ▁큰 ▁편이여가지구,20대,DKSR20000890.json,9,test
7,▁그래도 ▁한 ▁백칠십 오 에서 ▁팔 ▁정도면,▁그래도 ▁한 ▁백칠십 오 에서 ▁팔 ▁정도면,20대,DKSR20000890.json,5,test
8,▁딱 ▁좋은 ▁거 ▁같고 ▁일단 ▁그게 ▁겉으로 ▁보는 ▁이상형 ?,▁딱 ▁좋은 ▁거 ▁같고 ▁일단 ▁그게 ▁겉으로 ▁보는 ▁이상형 ?,20대,DKSR20000890.json,9,test
12,▁성격이 ▁일단 ▁제가 ▁쫌 ▁막 ▁드 센 ▁건 ▁아닌데,▁성격이 ▁일단 ▁제가 ▁조금 ▁막 ▁드 센 ▁건 ▁아닌데,20대,DKSR20000890.json,8,test
14,▁그래서 ▁성격을 ▁주로 ▁보는 ▁편인 ▁것 ▁같아요 .,▁그래서 ▁성격을 ▁주로 ▁보는 ▁편인 ▁것 ▁같아요 .,20대,DKSR20000890.json,7,test
...,...,...,...,...,...,...
229489,▁제일 ▁별로 였던 ▁쇼핑 ▁제일 ▁좋았던 ▁쇼핑이 ▁어떤 ▁게 ▁있으세요 ?,▁제일 ▁별로 였던 ▁쇼핑 ▁제일 ▁좋았던 ▁쇼핑이 ▁어떤 ▁게 ▁있으세요 ?,20대,DKSR20001736.json,9,test
229490,▁제일 ▁별로 였던 ▁거는 ▁그 ~ ▁동남아 ▁갔을 ▁때,▁제일 ▁별로 였던 ▁거는 ▁그 ~ ▁동남아 ▁갔을 ▁때,20대,DKSR20001736.json,7,test
229491,▁그 ~ ▁자전거 ▁같은 ▁거 ▁인력 거 ▁같은 ▁거 ?,▁그 ~ ▁자전거 ▁같은 ▁거 ▁인력 거 ▁같은 ▁거 ?,20대,DKSR20001736.json,7,test
229493,▁뒤에 ▁내가 ▁타는 ▁거 ▁그런 ▁거 ▁뭐라 ▁하는지 ▁모르겠는데,▁뒤에 ▁내가 ▁타는 ▁거 ▁그런 ▁거 ▁뭐라 ▁하는지 ▁모르겠는데,20대,DKSR20001736.json,9,test


In [135]:
spm.SentencePieceTrainer.train(input='../datas/output/gyeongsang_dialect_data_text.txt', model_prefix='gyeongsang_sp', vocab_size=30000)

In [136]:
sp_tokenizer = spm.SentencePieceProcessor(model_file='../model_storage/stan-GS_SP/gyeongsang_test_sp.model')

In [137]:
df.loc[df.index, "방언"] = [sp_tokenizer.decode(sentence.split()) for sentence in df["방언"]]
df.loc[df.index, "표준어"] = [sp_tokenizer.decode(sentence.split()) for sentence in df["표준어"]]

df

,방언,표준어,연령대,출처 파일,길이,셋
3,정해놓은 건 없는데 일단 제가 키가 되게 큰 편이여가지구,정해놓은 건 없는데 일단 제가 키가 되게 큰 편이여가지구,20대,DKSR20000890.json,9,test
7,그래도 한 백칠십오에서 팔 정도면,그래도 한 백칠십오에서 팔 정도면,20대,DKSR20000890.json,5,test
8,딱 좋은 거 같고 일단 그게 겉으로 보는 이상형?,딱 좋은 거 같고 일단 그게 겉으로 보는 이상형?,20대,DKSR20000890.json,9,test
12,성격이 일단 제가 쫌 막 드센 건 아닌데,성격이 일단 제가 조금 막 드센 건 아닌데,20대,DKSR20000890.json,8,test
14,그래서 성격을 주로 보는 편인 것 같아요.,그래서 성격을 주로 보는 편인 것 같아요.,20대,DKSR20000890.json,7,test
...,...,...,...,...,...,...
229489,제일 별로였던 쇼핑 제일 좋았던 쇼핑이 어떤 게 있으세요?,제일 별로였던 쇼핑 제일 좋았던 쇼핑이 어떤 게 있으세요?,20대,DKSR20001736.json,9,test
229490,제일 별로였던 거는 그~ 동남아 갔을 때,제일 별로였던 거는 그~ 동남아 갔을 때,20대,DKSR20001736.json,7,test
229491,그~ 자전거 같은 거 인력거 같은 거?,그~ 자전거 같은 거 인력거 같은 거?,20대,DKSR20001736.json,7,test
229493,뒤에 내가 타는 거 그런 거 뭐라 하는지 모르겠는데,뒤에 내가 타는 거 그런 거 뭐라 하는지 모르겠는데,20대,DKSR20001736.json,9,test


In [138]:
sp_tokenizer = spm.SentencePieceProcessor(model_file='./gyeongsang_sp.model')

code_result_dialect = sp_tokenizer.encode(df["방언"].to_list(), out_type=str)
code_result_standard = sp_tokenizer.encode(df["표준어"].to_list(), out_type=str)

In [139]:
df.loc[df.index, "방언"] = [' '.join(res) for res in code_result_dialect]
df["방언"]

3           ▁정해놓 은 ▁건 ▁없는데 ▁일단 ▁제가 ▁키가 ▁되게 ▁큰 ▁편이 여가지구
7                            ▁그래도 ▁한 ▁백칠십 오 에서 ▁팔 ▁정도면
8                ▁딱 ▁좋은 ▁거 ▁같고 ▁일단 ▁그게 ▁겉으로 ▁보는 ▁이상형 ?
12                     ▁성격이 ▁일단 ▁제가 ▁쫌 ▁막 ▁드 센 ▁건 ▁아닌데
14                     ▁그래서 ▁성격을 ▁주로 ▁보는 ▁편인 ▁것 ▁같아요 .
                              ...                     
229489     ▁제일 ▁별로 였던 ▁쇼핑 ▁제일 ▁좋았던 ▁쇼핑이 ▁어떤 ▁게 ▁있으세요 ?
229490                 ▁제일 ▁별로 였던 ▁거는 ▁그 ~ ▁동남아 ▁갔을 ▁때
229491                 ▁그 ~ ▁자전거 ▁같은 ▁거 ▁인력 거 ▁같은 ▁거 ?
229493           ▁뒤에 ▁내가 ▁타는 ▁거 ▁그런 ▁거 ▁뭐라 ▁하는지 ▁모르겠는데
229496    ▁그거 ▁타가지고 ▁바가지 를 ▁써가지고 ▁그 ▁싸웠던 ▁게 ▁생각 납 니다 .
Name: 방언, Length: 89511, dtype: object

In [140]:
df.loc[df.index, "표준어"] = [' '.join(res) for res in code_result_standard]
df["표준어"]

3           ▁정해놓 은 ▁건 ▁없는데 ▁일단 ▁제가 ▁키가 ▁되게 ▁큰 ▁편이 여가지구
7                            ▁그래도 ▁한 ▁백칠십 오 에서 ▁팔 ▁정도면
8                ▁딱 ▁좋은 ▁거 ▁같고 ▁일단 ▁그게 ▁겉으로 ▁보는 ▁이상형 ?
12                    ▁성격이 ▁일단 ▁제가 ▁조금 ▁막 ▁드 센 ▁건 ▁아닌데
14                     ▁그래서 ▁성격을 ▁주로 ▁보는 ▁편인 ▁것 ▁같아요 .
                              ...                     
229489     ▁제일 ▁별로 였던 ▁쇼핑 ▁제일 ▁좋았던 ▁쇼핑이 ▁어떤 ▁게 ▁있으세요 ?
229490                 ▁제일 ▁별로 였던 ▁거는 ▁그 ~ ▁동남아 ▁갔을 ▁때
229491                 ▁그 ~ ▁자전거 ▁같은 ▁거 ▁인력 거 ▁같은 ▁거 ?
229493           ▁뒤에 ▁내가 ▁타는 ▁거 ▁그런 ▁거 ▁뭐라 ▁하는지 ▁모르겠는데
229496    ▁그거 ▁타가지고 ▁바가지 를 ▁써가지고 ▁그 ▁싸웠던 ▁게 ▁생각 납 니다 .
Name: 표준어, Length: 89511, dtype: object

In [141]:
ori_df = pd.read_csv("../datas/output/gyeongsang_dialect_SentencePiece_integration.csv", index_col=0)

ori_df[ori_df["셋"] == "test"] = df
ori_df[ori_df["셋"] == "test"]

,방언,표준어,연령대,출처 파일,길이,셋
3,▁정해놓 은 ▁건 ▁없는데 ▁일단 ▁제가 ▁키가 ▁되게 ▁큰 ▁편이 여가지구,▁정해놓 은 ▁건 ▁없는데 ▁일단 ▁제가 ▁키가 ▁되게 ▁큰 ▁편이 여가지구,20대,DKSR20000890.json,9,test
7,▁그래도 ▁한 ▁백칠십 오 에서 ▁팔 ▁정도면,▁그래도 ▁한 ▁백칠십 오 에서 ▁팔 ▁정도면,20대,DKSR20000890.json,5,test
8,▁딱 ▁좋은 ▁거 ▁같고 ▁일단 ▁그게 ▁겉으로 ▁보는 ▁이상형 ?,▁딱 ▁좋은 ▁거 ▁같고 ▁일단 ▁그게 ▁겉으로 ▁보는 ▁이상형 ?,20대,DKSR20000890.json,9,test
12,▁성격이 ▁일단 ▁제가 ▁쫌 ▁막 ▁드 센 ▁건 ▁아닌데,▁성격이 ▁일단 ▁제가 ▁조금 ▁막 ▁드 센 ▁건 ▁아닌데,20대,DKSR20000890.json,8,test
14,▁그래서 ▁성격을 ▁주로 ▁보는 ▁편인 ▁것 ▁같아요 .,▁그래서 ▁성격을 ▁주로 ▁보는 ▁편인 ▁것 ▁같아요 .,20대,DKSR20000890.json,7,test
...,...,...,...,...,...,...
229489,▁제일 ▁별로 였던 ▁쇼핑 ▁제일 ▁좋았던 ▁쇼핑이 ▁어떤 ▁게 ▁있으세요 ?,▁제일 ▁별로 였던 ▁쇼핑 ▁제일 ▁좋았던 ▁쇼핑이 ▁어떤 ▁게 ▁있으세요 ?,20대,DKSR20001736.json,9,test
229490,▁제일 ▁별로 였던 ▁거는 ▁그 ~ ▁동남아 ▁갔을 ▁때,▁제일 ▁별로 였던 ▁거는 ▁그 ~ ▁동남아 ▁갔을 ▁때,20대,DKSR20001736.json,7,test
229491,▁그 ~ ▁자전거 ▁같은 ▁거 ▁인력 거 ▁같은 ▁거 ?,▁그 ~ ▁자전거 ▁같은 ▁거 ▁인력 거 ▁같은 ▁거 ?,20대,DKSR20001736.json,7,test
229493,▁뒤에 ▁내가 ▁타는 ▁거 ▁그런 ▁거 ▁뭐라 ▁하는지 ▁모르겠는데,▁뒤에 ▁내가 ▁타는 ▁거 ▁그런 ▁거 ▁뭐라 ▁하는지 ▁모르겠는데,20대,DKSR20001736.json,9,test


In [142]:
ori_df.to_csv("../datas/output/gyeongsang_dialect_SentencePiece_integration.csv")